In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import path
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy
import numpy as np
import netCDF4 as nc
np.set_printoptions(threshold=100000)
from shapely.geometry import Polygon, Point, MultiPoint, LineString, LinearRing
from shapely.ops import cascaded_union, unary_union, transform
import datetime
import math
from scipy.ndimage.interpolation import shift
import scipy.interpolate as si
import shapely.wkt
from shapely.validation import explain_validity
import xarray as xr
import seaborn as sns
from my_functions import sat_vap_press, vap_press, hot_dry_windy, haines
from joblib import Parallel, delayed
import multiprocessing
from os.path import exists
import rasterio
from rasterio.windows import get_data_window,Window, from_bounds
from rasterio.plot import show
from itertools import product

from timezonefinder import TimezoneFinder
import pytz

In [11]:
#process features using my polygons
sit209=pd.read_csv('../Query2.txt')
sit209.columns = ['INC209R_IDENTIFIER','INCIDENT_NAME','REPORT_FROM_DATE','REPORT_TO_DATE',
              'RESTYP_IDENTIFIER', 'RESOURCE_QUANTITY','RESOURCE_PERSONNEL','CODE_NAME',
              'INC_IDENTIFIER','PCT_CONTAINED_COMPLETED'] 

#all fires
#fire_incidents = ['BOBCAT', 'DOLAN', 'HOLIDAY FARM','CREEK', 'LAKE', 'CAMERON PEAK', 'PINE GULCH', 'WILLIAMS FLATS', 'SHADY','PEDRO MOUNTAIN', 'WALKER', '204 COW']

#2020 fires
#fire_incidents = ['AUGUST COMPLEX','BOBCAT', 'DOLAN', 'HOLIDAY FARM','CREEK', 'LAKE', 'CAMERON PEAK', 'PINE GULCH']
#fire_incidents=['BOBCAT', 'DOLAN', 'HOLIDAY FARM','CREEK', 'LAKE', 'CAMERON PEAK', 'PINE GULCH']
#fire_incidents = ['BOBCAT']
#fire_incidents = ['AUGUST COMPLEX', 'CREEK', 'DOLAN','LAKE']
fire_incidents = ['AUGUST COMPLEX', 'BOBCAT','CREEK', 'DOLAN','LAKE',
                  'HOLIDAY FARM','CAMERON PEAK', 'PINE GULCH','EAST TROUBLESOME']

path_poly = '/data2/lthapa/ML_daily/fire_polygons/'
suffix_poly = 'Z_day_start.geojson'
start_time=12
for jj in range(len(fire_incidents)):
    fire_name = fire_incidents[jj].lower().replace(' ','_')
    print(path_poly+fire_name+'_VIIRS_daily_'+str(start_time)+suffix_poly)
    fire_daily = gpd.read_file(path_poly+fire_name+'_VIIRS_daily_'+str(start_time)+suffix_poly)
    print(fire_daily.crs)
    fire_daily=fire_daily.drop(columns=['Current Overpass'])
    fire_daily = fire_daily.drop(np.where(fire_daily['geometry']==None)[0])
    fire_daily['fire area (ha)'] = fire_daily['geometry'].area/10000 #hectares. from m2
    fire_daily.set_geometry(col='geometry', inplace=True) #designate the geometry column
    fire_daily = fire_daily.rename(columns={'Current Day':'UTC Day', 'Local Day': str(start_time)+ 'Z Start Day'})
    
    fire_daily = fire_daily.iloc[np.array(fire_daily['UTC Day'].values,dtype='datetime64')<=np.datetime64('2020-10-31'),:]
    #print(fire_daily)
    #merra
    #me = merra_timeseries(fire_daily, start_time)
    #print(me)
    #me.to_csv('./fire_features/'+fire_name+'_Daily_MERRA_'+str(start_time)+'Z_day_start.csv') #daily averages

    #hrrr
    #hrrr = hrrr_timeseries(fire_daily,start_time)
    #print(hrrr)
    #hrrr.to_csv('./fire_features/'+fire_name+'_Daily_HRRR_'+str(start_time)+'Z_day_start.csv') #daily averages
    
    #hrrr with average wind
    #hrrr = hrrr_timeseries_ws(fire_daily,start_time)
    #print(hrrr)
    #hrrr.to_csv('./fire_features/'+fire_name+'_Daily_HRRR_ws_'+str(start_time)+'Z_day_start.csv') #daily averages
    
    #rave
    ##rave=rave_timeseries(fire_daily,start_time)
    #print(rave)
    #rave.to_csv('./fire_features/'+fire_name+'_Daily_RAVE_'+str(start_time)+'Z_day_start.csv') #daily averages

    #pws
    #pws = pws_timeseries(fire_daily,start_time)
    #print(pws)
    #pws.to_csv('./fire_features/'+fire_name+'_Daily_PWS_'+str(start_time)+'Z_day_start.csv') #daily averages
   
    #hrrr_igbp
    #igbp =hrrr_igbp_timeseries(fire_daily,start_time)
    #print(igbp)
    #igbp.to_csv('./fire_features/'+fire_name+'_Daily_IGBP_'+str(start_time)+'Z_day_start.csv') #daily averages

    #esi 
    #esi = esi_timeseries(fire_daily,start_time)
    #print(esi)
    #esi.to_csv('./fire_features/'+fire_name+'_Daily_ESI_'+str(start_time)+'Z_day_start.csv') #daily averages
    
    #sit 209 data
    #resources = resources_timeseries(fire_daily,start_time,sit209)
    #print(resources)
    #resources.to_csv('./fire_features/'+fire_name+'_Daily_Resources_'+str(start_time)+'Z_day_start.csv')
    
    #fuel loading (takes 20 mins to process AC on the 1km grid)
    #fuel_loading = fuel_loading_timeseries(fire_daily,start_time)
    #print(fuel_loading)
    #fuel_loading.to_csv('./fire_features/'+fire_name+'_Daily_FUEL_LOADING_'+str(start_time)+'Z_day_start.csv') #daily averages
    
    #fire area
    #fire_area = fire_area_timeseries(fire_daily,start_time)
    #print(fire_area)
    #fire_area.to_csv('./fire_features/'+fire_name+'_Daily_FIRE_AREA_'+str(start_time)+'Z_day_start.csv') #daily averages

    #IMERG FWI
    #imerg_fwi = imerg_fwi_timeseries(fire_daily,12)
    #print(imerg_fwi)
    #imerg_fwi.to_csv('./fire_features/'+fire_name+'_Daily_IMERG_FWI_NoAve_'+str(start_time)+'Z_day_start.csv') #daily averages

    #imerg_fwi.to_csv('./fire_features/'+fire_name+'_Daily_IMERG_FWI_'+str(start_time)+'Z_day_start.csv') #daily averages

    #Fuel Loadings fwi
    #fuel_loading_fwi = fuel_loading_fwi_timeseries(fire_daily,12)
    #print(fuel_loading_fwi)
    #fuel_loading_fwi.to_csv('./fire_features/'+fire_name+'_Daily_FUEL_FWI_'+str(start_time)+'Z_day_start.csv') #daily averages

    
    #low res fuel loadings FWI
    #fuel_loading_fwi_lores = fuel_loading_fwi_lowres_timeseries(fire_daily,12)
    """tic = datetime.datetime.now()
    fuel_fwi_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (fire_daily.iloc[ii:ii+1],'FUEL_FWI_GRID_990M',0.05) 
                                 for ii in range(len(fire_daily)))
    toc = datetime.datetime.now()
    print(toc-tic)
    print([fuel_fwi_intersections[jj]['weights'].sum() for jj in range(len(fuel_fwi_intersections))])
    
    fire_fuel_fwi_intersection=gpd.GeoDataFrame(pd.concat(fuel_fwi_intersections, ignore_index=True))
    fire_fuel_fwi_intersection = fire_fuel_fwi_intersection.drop(columns='geometry')
    fire_fuel_fwi_intersection = fire_fuel_fwi_intersection.set_index(['12Z Start Day','row', 'col'])
    fire_fuel_fwi_intersection_xr = fire_fuel_fwi_intersection.to_xarray()

    #path_fuel_fwi = '/data2/lthapa/ML_daily/bobcat_fuel_fwi.nc'
    #path_fuel_fwi = '/data2/lthapa/ML_daily/bobcat_fuel_fwi_360m.nc'
    path_fuel_fwi = '/data2/lthapa/ML_daily/fuel_fwi_990m.nc'
    dat_fuel_fwi = xr.open_dataset(path_fuel_fwi) #map is fixed in time
    dat_fuel_fwi_daily = dat_fuel_fwi.expand_dims({'time': pd.to_datetime(fire_fuel_fwi_intersection_xr['12Z Start Day'].values)}) #the PWS expanded over all the days

    dat_fuel_fwi_sub_daily = dat_fuel_fwi_daily.sel(row = fire_fuel_fwi_intersection_xr['row'].values, 
                                        col = fire_fuel_fwi_intersection_xr['col'].values, method='nearest')
    #preallocate space for the output
    varis = ['day','Extreme_N', 'VeryHigh_N','High_N', 'Moderate_N', 'Low_N']
    df_loading = generate_df(varis, len(fire_daily))

    df_loading['day'] = fire_daily['12Z Start Day'].values
    for var in varis[1:len(varis)]:
        df_loading[var] = np.nansum(fire_fuel_fwi_intersection_xr['weights'].values*dat_fuel_fwi_sub_daily[var].values, axis=(1,2))

    print(df_loading)
    df_loading.to_csv('./fire_features/'+fire_name+'_Daily_COARSE990_FUEL_FWI_12Z_day_start.csv') #daily averages"""


/data2/lthapa/ML_daily/fire_polygons/august_complex_VIIRS_daily_12Z_day_start.geojson
epsg:3347
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200817.nc
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200818.nc
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200819.nc
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200820.nc
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200821.nc
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200822.nc
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200823.nc
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200824.nc
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200825.nc
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200826.nc
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200827.nc
/data2/lthapa

/home/lthapa/anaconda3/envs/ML_py/lib/python3.7/site-packages/xarray/coding/variables.py:141: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  condition |= data == fv


          day  IMERG.FINAL.v6_DC  IMERG.FINAL.v6_DMC  IMERG.FINAL.v6_FFMC  \
0  2020-08-16         951.252930          215.804047            98.260033   
1  2020-08-17         938.127686          118.552208            94.137016   
2  2020-08-18         911.440247           84.243073            94.634232   
3  2020-08-19         920.865845           90.447594            95.969124   
4  2020-08-20         930.258179           96.561996            96.109840   
5  2020-08-21         939.063293          101.288414            94.749496   
6  2020-08-22         919.697205           78.191063            90.529686   
7  2020-08-23         929.269714           84.570030            96.156723   
8  2020-08-24         895.490051           63.132023            91.106026   
9  2020-08-25         904.969238           69.399895            96.082993   
10 2020-08-26         914.262756           75.617607            96.593613   
11 2020-08-27         923.284790           81.447182            96.645149   

epsg:3347
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200906.nc
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200907.nc
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200908.nc
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200909.nc
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200910.nc
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200911.nc
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200912.nc
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200913.nc
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200914.nc
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200915.nc
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200916.nc
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200917.nc
/data2/lthapa/2020

/home/lthapa/anaconda3/envs/ML_py/lib/python3.7/site-packages/xarray/coding/variables.py:141: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  condition |= data == fv


...

          day  IMERG.FINAL.v6_DC  IMERG.FINAL.v6_DMC  IMERG.FINAL.v6_FFMC  \
0  2020-08-12         749.225220          141.517303            96.262459   
1  2020-08-13         758.133057          147.270584            96.313881   
2  2020-08-14         767.352234          153.760300            97.542366   
3  2020-08-15         776.487000          160.115860            97.594215   
4  2020-08-16         785.525696          165.872437            96.456612   
5  2020-08-17         794.461060          171.481155            96.508102   
6  2020-08-18         755.780823           87.702477            89.572495   
7  2020-08-19         675.380920           50.307518            87.857712   
8  2020-08-20         625.735596           32.289288            84.063179   
9  2020-08-21         634.388428           36.633682            91.446838   
10 2020-08-22         643.101135           39.112026            89.797279   
11 2020-08-23         651.732239           44.021599            92.055664   

epsg:3347
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200801.nc
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200802.nc
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200803.nc
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200804.nc
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200805.nc
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200806.nc
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200807.nc
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200808.nc
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200809.nc
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200810.nc
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200811.nc
/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.20200812.nc
/data2/lthapa/2020

/home/lthapa/anaconda3/envs/ML_py/lib/python3.7/site-packages/xarray/coding/variables.py:141: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  condition |= data == fv


          day  IMERG.FINAL.v6_DC  IMERG.FINAL.v6_DMC  IMERG.FINAL.v6_FFMC  \
0  2020-07-31         710.698486          110.369064            96.046875   
1  2020-08-01         719.413574          115.450211            94.076653   
2  2020-08-02         728.318604          121.429314            96.584511   
3  2020-08-03         736.853882          127.055000            96.636040   
4  2020-08-04         745.227539          132.366714            96.687592   
5  2020-08-05         753.510437          137.794479            96.739159   
6  2020-08-06         761.879578          143.163986            96.790741   
7  2020-08-07         770.522644          148.597641            96.806549   
8  2020-08-08         779.317566          154.338837            96.858154   
9  2020-08-09         788.083435          160.386490            97.290627   
10 2020-08-10         796.813660          166.193298            97.342392   
11 2020-08-11         805.603699          171.681702            96.867897   

/home/lthapa/anaconda3/envs/ML_py/lib/python3.7/site-packages/xarray/coding/variables.py:141: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  condition |= data == fv


In [59]:
#all fires
#fire_incidents = ['BOBCAT', 'DOLAN', 'HOLIDAY FARM','CREEK', 'LAKE', 'CAMERON PEAK', 'PINE GULCH', 'WILLIAMS FLATS', 'SHADY','PEDRO MOUNTAIN', 'WALKER', '204 COW']

#2020 fires
fire_incidents = ['AUGUST COMPLEX','BOBCAT', 'DOLAN', 'HOLIDAY FARM','CREEK', 'LAKE', 
                 'CAMERON PEAK', 'PINE GULCH','EAST TROUBLESOME']
#fire_incidents = ['AUGUST COMPLEX', 'DOLAN', 'HOLIDAY FARM','CREEK', 'CAMERON PEAK', 'PINE GULCH','EAST TROUBLESOME']
#fire_incidents=['BOBCAT', 'DOLAN', 'HOLIDAY FARM','CREEK', 'LAKE', 'CAMERON PEAK', 'PINE GULCH']
#fire_incidents = ['LAKE','PINE GULCH']
#fire_incidents = ['AUGUST COMPLEX', 'CREEK', 'DOLAN','LAKE']


path_poly = '/data2/lthapa/ML_daily/fire_polygons/'
suffix_poly = 'Z_day_start.geojson'
start_time=12
for jj in range(len(fire_incidents)):
    fire_name = fire_incidents[jj].lower().replace(' ','_')
    print(path_poly+fire_name+'_VIIRS_daily_'+str(start_time)+suffix_poly)
    fire_daily = gpd.read_file(path_poly+fire_name+'_VIIRS_daily_'+str(start_time)+suffix_poly)
    print(fire_daily.crs)
    fire_daily=fire_daily.drop(columns=['Current Overpass'])
    fire_daily = fire_daily.drop(np.where(fire_daily['geometry']==None)[0])
    fire_daily['fire area (ha)'] = fire_daily['geometry'].area/10000 #hectares. from m2
    fire_daily.set_geometry(col='geometry', inplace=True) #designate the geometry column
    fire_daily = fire_daily.rename(columns={'Current Day':'UTC Day', 'Local Day': str(start_time)+ 'Z Start Day'})
    
    fire_daily = fire_daily.iloc[np.array(fire_daily['UTC Day'].values,dtype='datetime64')<=np.datetime64('2020-10-31'),:]
    
    #SLOPE
    #slope = slope_timeseries_coarse(fire_daily,start_time)
    #print(slope)
    #slope.to_csv('./fire_features/'+fire_name+'_Daily_slope_990m_'+str(start_time)+'Z_day_start.csv') #daily averages
    
    #PWS
    #pws = pws_timeseries(fire_daily,start_time)
    #print(pws)
    
    #SMOPS
    #smops = smops_timeseries(fire_daily, start_time)
    #print(smops)
    #smops.to_csv('./fire_features/'+fire_name+'_Daily_SMOPS_'+str(start_time)+'Z_day_start.csv') #daily averages
    
    #NCAR moisture
    #ncar = ncar_timeseries(fire_daily, start_time)
    #print(ncar)
    #ncar.to_csv('./fire_features/'+fire_name+'_Daily_NCAR_Moisture_'+str(start_time)+'Z_day_start.csv') #daily averages
    
    #HRRR-derived continuous Haines Index
    chi = hrrr_timeseries_chi(fire_daily,start_time)
    print(chi)
    chi.to_csv('./fire_features/'+fire_name+'_Daily_CHI_'+str(start_time)+'Z_day_start.csv') #daily averages


/data2/lthapa/ML_daily/fire_polygons/august_complex_VIIRS_daily_12Z_day_start.geojson
epsg:3347
0:03:58.491102
['2020-08-15T12:00:00.000000000' '2020-08-16T12:00:00.000000000'
 '2020-08-17T12:00:00.000000000' '2020-08-18T12:00:00.000000000'
 '2020-08-19T12:00:00.000000000' '2020-08-20T12:00:00.000000000'
 '2020-08-21T12:00:00.000000000' '2020-08-22T12:00:00.000000000'
 '2020-08-23T12:00:00.000000000' '2020-08-24T12:00:00.000000000'
 '2020-08-25T12:00:00.000000000' '2020-08-26T12:00:00.000000000'
 '2020-08-27T12:00:00.000000000' '2020-08-28T12:00:00.000000000'
 '2020-08-29T12:00:00.000000000' '2020-08-30T12:00:00.000000000'
 '2020-08-31T12:00:00.000000000' '2020-09-01T12:00:00.000000000'
 '2020-09-02T12:00:00.000000000' '2020-09-03T12:00:00.000000000'
 '2020-09-04T12:00:00.000000000' '2020-09-05T12:00:00.000000000'
 '2020-09-06T12:00:00.000000000' '2020-09-07T12:00:00.000000000'
 '2020-09-08T12:00:00.000000000' '2020-09-09T12:00:00.000000000'
 '2020-09-10T12:00:00.000000000' '2020-09-11

epsg:3347
0:00:21.153836
['2020-09-04T12:00:00.000000000' '2020-09-05T12:00:00.000000000'
 '2020-09-06T12:00:00.000000000' '2020-09-07T12:00:00.000000000'
 '2020-09-08T12:00:00.000000000' '2020-09-09T12:00:00.000000000'
 '2020-09-10T12:00:00.000000000' '2020-09-11T12:00:00.000000000'
 '2020-09-12T12:00:00.000000000' '2020-09-13T12:00:00.000000000'
 '2020-09-14T12:00:00.000000000' '2020-09-15T12:00:00.000000000'
 '2020-09-16T12:00:00.000000000' '2020-09-17T12:00:00.000000000'
 '2020-09-18T12:00:00.000000000' '2020-09-19T12:00:00.000000000'
 '2020-09-20T12:00:00.000000000' '2020-09-21T12:00:00.000000000'
 '2020-09-22T12:00:00.000000000' '2020-09-23T12:00:00.000000000'
 '2020-09-24T12:00:00.000000000' '2020-09-25T12:00:00.000000000'
 '2020-09-26T12:00:00.000000000' '2020-09-27T12:00:00.000000000'
 '2020-09-28T12:00:00.000000000' '2020-09-29T12:00:00.000000000'
 '2020-09-30T12:00:00.000000000' '2020-10-01T12:00:00.000000000'
 '2020-10-02T12:00:00.000000000' '2020-10-03T12:00:00.000000000'


          day    temp_850    temp_700   dewpt_850        ca        cb  \
0  2020-08-18  300.248747  288.087109  279.847299  4.080805  5.400235   
1  2020-08-19  299.203122  287.268415  274.463711  3.967365  6.123238   
2  2020-08-20  298.245636  286.620518  267.372697  3.812556  6.713045   
3  2020-08-21  297.931944  286.708215  267.936359  3.611871  6.695698   
4  2020-08-22  298.049997  287.222779  270.776050  3.413613  6.276513   
5  2020-08-23  296.204734  285.732347  277.011371  3.236189  4.991740   
6  2020-08-24  293.282640  283.785313  284.154867  2.748652  2.042585   
7  2020-08-25  290.861396  284.749499  285.739901  1.055946  0.707169   
8  2020-08-26  294.069142  283.620446  274.399664  3.224340  4.730271   
9  2020-08-27  293.970125  284.414047  266.047999  2.778026  6.567889   
10 2020-08-28  295.021199  283.977153  273.379289  3.522028  5.604517   
11 2020-08-29  294.611188  284.114649  266.759384  3.248267  6.325214   
12 2020-08-30  296.068276  285.426446  266.409055  

          day    temp_850    temp_700   dewpt_850        ca        cb  \
0  2020-09-04  300.758175  288.984464  277.297045  3.886857  5.880523   
1  2020-09-05  301.451971  288.083095  275.623437  4.684441  6.299606   
2  2020-09-06  302.838240  288.770187  275.196989  5.034047  6.532313   
3  2020-09-07  301.546324  287.760937  272.051989  4.892692  6.657098   
4  2020-09-08  297.233504  286.424912  266.989618  3.404288  6.535421   
5  2020-09-09  293.559892  284.674402  266.465675  2.442739  6.410996   
6  2020-09-10  291.933614  282.536085  267.836121  2.698768  6.015590   
7  2020-09-11  293.433818  283.653969  273.339819  2.889917  4.965580   
8  2020-09-12  295.432235  284.788763  268.717977  3.321742  6.341393   
9  2020-09-13  295.848117  284.299892  266.839633  3.774116  6.686998   
10 2020-09-14  295.263178  283.676006  271.641179  3.793589  5.918243   
11 2020-09-15  294.641169  283.792983  278.450321  3.424096  4.191033   
12 2020-09-16  294.594989  284.119939  275.234656  

          day    temp_850    temp_700   dewpt_850        ca        cb  \
0  2020-08-12  297.770508  287.071381  277.796173  3.349580  5.200689   
1  2020-08-13  299.204293  288.475034  272.192370  3.364628  6.374402   
2  2020-08-14  298.530332  287.765406  272.851516  3.382458  6.168685   
3  2020-08-15  298.357083  287.587689  274.372829  3.384716  5.964907   
4  2020-08-16  298.915136  288.015799  278.739098  3.449668  5.040695   
5  2020-08-17  300.010125  289.187324  280.979017  3.411389  4.937385   
6  2020-08-18  300.852198  289.998091  280.823813  3.427048  5.211331   
7  2020-08-19  299.651950  288.982104  281.812445  3.334936  4.830655   
8  2020-08-20  298.165221  287.465209  283.911821  3.350003  3.741677   
9  2020-08-21  299.295641  288.528791  280.790540  3.383434  4.746661   
10 2020-08-22  300.381702  289.647337  276.419142  3.367189  5.929236   
11 2020-08-23  300.905490  290.111247  280.319554  3.397113  5.348443   
12 2020-08-24  300.589593  289.974736  282.924918  

epsg:3347
0:00:44.426725
['2020-10-12T12:00:00.000000000' '2020-10-13T12:00:00.000000000'
 '2020-10-14T12:00:00.000000000' '2020-10-15T12:00:00.000000000'
 '2020-10-16T12:00:00.000000000' '2020-10-17T12:00:00.000000000'
 '2020-10-18T12:00:00.000000000' '2020-10-19T12:00:00.000000000'
 '2020-10-20T12:00:00.000000000' '2020-10-21T12:00:00.000000000'
 '2020-10-22T12:00:00.000000000' '2020-10-23T12:00:00.000000000'
 '2020-10-24T12:00:00.000000000']
['2020-10-13 12:00:00', '2020-10-14 12:00:00', '2020-10-15 12:00:00', '2020-10-16 12:00:00', '2020-10-17 12:00:00', '2020-10-18 12:00:00', '2020-10-19 12:00:00', '2020-10-20 12:00:00', '2020-10-21 12:00:00', '2020-10-22 12:00:00', '2020-10-23 12:00:00', '2020-10-24 12:00:00']
          day    temp_850    temp_700   dewpt_850        ca        cb  \
0  2020-10-13  290.756348  280.604279  270.023712  3.076018  5.244063   
1  2020-10-14  287.610833  276.122180  271.756988  3.744326  4.252255   
2  2020-10-15  280.115629  269.766440  267.356309  3.17

In [11]:
# the  rest
fire_fuel_fwi_intersection=gpd.GeoDataFrame(pd.concat(fuel_fwi_intersections, ignore_index=True))
fire_fuel_fwi_intersection = fire_fuel_fwi_intersection.drop(columns='geometry')
fire_fuel_fwi_intersection = fire_fuel_fwi_intersection.set_index(['12Z Start Day','row', 'col'])
fire_fuel_fwi_intersection_xr = fire_fuel_fwi_intersection.to_xarray()

#path_fuel_fwi = '/data2/lthapa/ML_daily/bobcat_fuel_fwi.nc'
#path_fuel_fwi = '/data2/lthapa/ML_daily/bobcat_fuel_fwi_360m.nc'
path_fuel_fwi = '/data2/lthapa/ML_daily/bobcat_fuel_fwi_990m.nc'
dat_fuel_fwi = xr.open_dataset(path_fuel_fwi) #map is fixed in time
dat_fuel_fwi_daily = dat_fuel_fwi.expand_dims({'time': pd.to_datetime(fire_fuel_fwi_intersection_xr['12Z Start Day'].values)}) #the PWS expanded over all the days

dat_fuel_fwi_sub_daily = dat_fuel_fwi_daily.sel(row = fire_fuel_fwi_intersection_xr['row'].values, 
                                    col = fire_fuel_fwi_intersection_xr['col'].values, method='nearest')
#preallocate space for the output
varis = ['day','Extreme_N', 'VeryHigh_N','High_N', 'Moderate_N', 'Low_N']
df_loading = generate_df(varis, len(fire_daily))
    
df_loading['day'] = fire_daily['12Z Start Day'].values
for var in varis[1:len(varis)]:
    df_loading[var] = np.nansum(fire_fuel_fwi_intersection_xr['weights'].values*dat_fuel_fwi_sub_daily[var].values, axis=(1,2))

print(df_loading)

df_loading.to_csv('bobcat_Daily_COARSE990_FUEL_FWI_12Z_day_start.csv')

           day  Extreme_N  VeryHigh_N     High_N  Moderate_N     Low_N
0   2020-09-05   3.802369    2.954377   2.257590    1.512732  0.831968
1   2020-09-06  10.520819    8.409648   6.452405    3.214916  1.684422
2   2020-09-07  14.703652   11.673446   9.004202    4.428434  2.394698
3   2020-09-08  15.215874   12.047310   9.238419    5.073312  2.716493
4   2020-09-09  15.961910   12.661959   9.780726    4.744564  2.581641
5   2020-09-10  19.559030   15.242757  11.910685    5.787808  3.386004
6   2020-09-11  17.272519   13.492138  10.478735    5.450689  3.108834
7   2020-09-12  16.116388   12.540743   9.745848    5.209715  2.995232
8   2020-09-13  17.089573   13.395030  10.437410    4.996910  2.866916
9   2020-09-14  12.358782    9.672784   7.535705    3.671983  2.110451
10  2020-09-15  17.698871   13.698900  10.771029    5.097207  3.084654
11  2020-09-16  25.273094   19.225274  15.258541    7.460044  4.770565
12  2020-09-17  18.288434   14.140546  11.046520    5.820753  3.437307
13  20

In [ ]:
# process features using the Chen polygons
#2020 fires
#fire_incidents = ['LAKE', 'DOLAN','BOBCAT','AUGUST COMPLEX', 'CREEK']
fire_incidents = ['LAKE']

start_time=12
path_poly = '/data2/lthapa/ML_daily/fire_polygons/'
suffix_poly = '_Chen_Poly_Perim_Perp.csv'

for jj in range(len(fire_incidents)):
    fire_name = fire_incidents[jj].lower().replace(' ','_')
    #print(path_poly+fire_name+suffix_poly)
    fire = pd.read_csv(path_poly+fire_name+suffix_poly)
    fire=fire.dropna()
    
    for c in ['perimeter','polygon','fireline_split','fireline_perpindicular']:
        fire[c] = fire[c].apply(shapely.wkt.loads) #read in the polygons as shapes
    fire['time'] = fire['time'].astype('datetime64')

    
    #get the polygon dataframe set up
    fire_poly = gpd.GeoDataFrame(fire[['time','polygon']],geometry='polygon')
    fire_poly[str(start_time)+ 'Z Start Day'] = (pd.to_datetime(fire_poly['time'].values)-datetime.timedelta(hours=12)).strftime('%Y-%m-%d')

    fire_poly['polygon'] = fire_poly['polygon'].buffer(0.001763092448) #need to buffer because we saved the points, which have no area
    fire_poly = fire_poly.set_crs('EPSG:4326') #set the CRS to lat/lon
    fire_poly = fire_poly.to_crs('EPSG:3347') #transform to LCC

    #dissolve to check daily
    #fire_poly = fire_poly.dissolve(by='12Z Start Day').reset_index()

    fire_poly['UTC Day'] = (pd.to_datetime(fire_poly['time'].values)).strftime('%Y-%m-%d')
    fire_poly['fire area (ha)'] = fire_poly['polygon'].area/10000 #hectares. from m2

    #print(fire_poly)
    
    #merra
    #me = merra_timeseries(fire_poly, start_time)
    #print(me)
    #me.to_csv('./fire_features/'+fire_name+'_Daily_MERRA_'+str(start_time)+'Z_day_start_Chen.csv') #daily averages
    
    #hrrr with average wind
    #hrrr = hrrr_timeseries_sub_daily(fire_poly,start_time,12)
    #print(hrrr)
        
    #get the perpindicular lines set up
    fire_split = gpd.GeoDataFrame(fire[['time','fireline_split']],geometry='fireline_split')
    fire_split[str(start_time)+ 'Z Start Day'] = (pd.to_datetime(fire_split['time'].values)-datetime.timedelta(hours=12)).strftime('%Y-%m-%d')
    fire_split = fire_split.iloc[np.array(fire_poly['UTC Day'].values,dtype='datetime64')<=np.datetime64('2020-10-31'),:]
 
    #print(fire_perp)
    #print(fire_poly)
    fire_perim  = gpd.GeoDataFrame(fire[['time','perimeter']],geometry='perimeter')
    fire_perim = fire_perim.iloc[np.array(fire_poly['UTC Day'].values,dtype='datetime64')<=np.datetime64('2020-10-31'),:]
    
    fire_poly = fire_poly.iloc[np.array(fire_poly['UTC Day'].values,dtype='datetime64')<=np.datetime64('2020-10-31'),:]

    #HRRR Directional Test
    #hrrr_directional_test = hrrr_directional_hdw_test(fire_split,fire_poly,fire_perim,start_time, 12)
    #print(hrrr_directional_test)
    #hrrr_directional_test.to_csv('./fire_features/'+fire_name+'_SubDaily_HRRR_Directional_'+str(start_time)+'Z_day_start_Chen.csv') #daily averages

    #rave daily
    #fire_poly_dissolved = fire_poly.dissolve(by=str(start_time)+'Z Start Day').reset_index()
    #print(fire_poly_dissolved[str(start_time)+'Z Start Day'].values+'T'+str(start_time)+'00:00')
    #fire_poly_dissolved.loc[:,'time'] = [fire_poly_dissolved[str(start_time)+'Z Start Day'].values[kk]+
    #                                     'T'+str(start_time)+':00:00' 
    #                                    for kk in range(len(fire_poly_dissolved))]
    #print(rave_timeseries(fire_poly_dissolved,12, 24))
    
    #rave_subdaily
    #print(fire_poly)
    #rave_subdaily = rave_timeseries(fire_poly,12, 12)
    #print(rave_subdaily)
    #rave_subdaily.to_csv('./fire_features/'+fire_name+'_SubDaily_RAVE_'+str(start_time)+'Z_day_start_Chen.csv') #daily averages

    #HRRR Directional HOURLY
    hrrr_hourly = hrrr_directional_hourly(fire_split,fire_poly,fire_perim,start_time, 12)
    print(hrrr_hourly)
    hrrr_hourly.to_csv('./fire_features/'+fire_name+'_Hourly_HRRR_Directional_Daily_'+str(start_time)+'Z_day_start_Chen.csv') #daily averages

In [ ]:
print(hrrr_hourly.columns)
hrrr_hourly.to_csv('./fire_features/'+fire_name+'_Hourly_HRRR_Directional_Daily_'+str(start_time)+'Z_day_start_Chen.csv') #daily averages

In [ ]:
pd.read_csv('FWI_Category_Fuel_Loadings.csv')

## Dataset-Dependent Helper Functions

### NCAR Moisture Data

In [27]:
def ncar_timeseries(df, day_start_hour):
    varis_ncar = ['day','FMCG2D','FMCGLH2D']
    df_ncar = generate_df(varis_ncar, len(df))
    #do the intersection, in parallel
    ncar_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (df.iloc[ii:ii+1],'NCAR_MOISTURE_GRID',0.5) 
                                 for ii in range(len(df)))
    #print([ncar_intersections[jj]['weights'].sum() for jj in range(len(ncar_intersections))])

    
    fire_ncar_intersection=gpd.GeoDataFrame(pd.concat(ncar_intersections, ignore_index=True))
    fire_ncar_intersection.set_geometry(col='geometry')  
    fire_ncar_intersection = fire_ncar_intersection.set_index([str(day_start_hour)+'Z Start Day', 'row', 'col'])
    fire_ncar_intersection=fire_ncar_intersection[~fire_ncar_intersection.index.duplicated()]

    fire_ncar_intersection_xr = fire_ncar_intersection.to_xarray()
    #print(fire_ncar_intersection)
    #load in rave data associated with the fire
    times = pd.date_range(np.datetime64(df[str(day_start_hour)+ 'Z Start Day'].iloc[0]),
                        np.datetime64(df[str(day_start_hour)+ 'Z Start Day'].iloc[len(df)-1])+
                        np.timedelta64(1,'D'))
    ncar_filenames,times_back_used = make_file_namelist(times,'/data2/lthapa/YYYY/FMC/fmc_YYYYMMDD_20Z.nc')

    
    dat_ncar = xr.open_mfdataset(ncar_filenames,concat_dim='Time',combine='nested',compat='override', coords='all')
    dat_ncar = dat_ncar.assign_coords({'Time': times_back_used}) #assign coords so we can select in time
    #print(dat_ncar)
    
    
    #select the locations and times we want
    dat_ncar_sub = dat_ncar.isel(south_north = fire_ncar_intersection_xr['row'].values.astype(int), 
                                 west_east = fire_ncar_intersection_xr['col'].values.astype(int)).sel(
                                 Time = pd.to_datetime(fire_ncar_intersection_xr[str(day_start_hour)+'Z Start Day'].values))#these should be lined up correctly
    
    df_ncar['day'].iloc[:] = pd.to_datetime(fire_ncar_intersection_xr[str(day_start_hour)+'Z Start Day'].values)
    for var in varis_ncar[1:]:
        df_ncar[var] = np.nansum(fire_ncar_intersection_xr['weights'].values*dat_ncar_sub[var].values, axis=(1,2))
        
    df_ncar.iloc[df_ncar['day']=='2020-09-09'] = [pd.date_range(np.datetime64('2020-09-09'),np.datetime64('2020-09-09')+np.timedelta64(0,'D')),
                                                           np.nan,np.nan]
    return df_ncar

### SMOPS Data

In [57]:
def smops_timeseries(df, day_start_hour):
    varis_smops = ['day','Blended_SM']
    df_smops = generate_df(varis_smops, len(df))
    #do the intersection, in parallel
    smops_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (df.iloc[ii:ii+1],'SMOPS_GRID',0.5) 
                                 for ii in range(len(df)))
    fire_smops_intersection=gpd.GeoDataFrame(pd.concat(smops_intersections, ignore_index=True))
    fire_smops_intersection.set_geometry(col='geometry')  
    fire_smops_intersection = fire_smops_intersection.set_index([str(day_start_hour)+'Z Start Day', 'row', 'col'])
    fire_smops_intersection=fire_smops_intersection[~fire_smops_intersection.index.duplicated()]

    fire_smops_intersection_xr = fire_smops_intersection.to_xarray()

    #load in rave data associated with the fire
    times = pd.date_range(np.datetime64(df[str(day_start_hour)+ 'Z Start Day'].iloc[0]),
                        np.datetime64(df[str(day_start_hour)+ 'Z Start Day'].iloc[len(df)-1])+
                        np.timedelta64(1,'D'))
    smops_filenames,times_back_used = make_file_namelist(times,'/data2/lthapa/YYYY/SMOPS/NPR_SMOPS_CMAP_DYYYYMMDD.nc')

    dat_smops = xr.open_mfdataset(smops_filenames,concat_dim='Time',combine='nested',compat='override', coords='all')
    dat_smops = dat_smops.assign_coords({'Time': times_back_used}) #assign coords so we can select in time
    print(dat_smops['Time'].values)
    #select the locations and times we want
    dat_smops_sub = dat_smops.isel(Latitude = fire_smops_intersection_xr['row'].values.astype(int), 
                    Longitude = fire_smops_intersection_xr['col'].values.astype(int)).sel(
                    Time = pd.to_datetime(fire_smops_intersection_xr[str(day_start_hour)+'Z Start Day'].values))#these should be lined up correctly
    
    df_smops['day'].iloc[:] = pd.to_datetime(fire_smops_intersection_xr[str(day_start_hour)+'Z Start Day'].values)
    for var in varis_smops[1:]:
        df_smops[var] = np.nansum(fire_smops_intersection_xr['weights'].values*dat_smops_sub[var].values, axis=(1,2))
    
    return df_smops

### Slope Data

In [59]:
#do for the 30m one
def slope_timeseries(df,day_start_hour):
    #print(df)
    tic = datetime.datetime.now()
    slope_intersections = Parallel(n_jobs=12)(delayed(calculate_intersection)
                                 (df.iloc[ii:ii+1],'BOBCAT_SLOPE_30M',0.05) 
                                 for ii in range(len(df)))
    toc = datetime.datetime.now()
    print(toc-tic)
    #print([slope_intersections[jj]['weights'].sum() for jj in range(len(slope_intersections))])

    
    fire_slope_intersection=gpd.GeoDataFrame(pd.concat(slope_intersections, ignore_index=True))
    print(len(fire_slope_intersection))
    
    #fire_fuel_fwi_intersection.set_geometry(col='geometry')
    fire_slope_intersection = fire_slope_intersection.drop(columns='geometry')
    fire_slope_intersection = fire_slope_intersection.set_index([str(day_start_hour)+ 'Z Start Day', 'row', 'col'])
    fire_slope_intersection_xr = fire_slope_intersection.to_xarray()
    toc = datetime.datetime.now()
    #print(fire_slope_intersection_xr)

    
    #load in slope data associated with the fire (it's only one dataset)
    
    #open the slope file
    path_slope = '/data2/lthapa/ML_daily/bobcat_slope_30m.nc'
    dat_slope = xr.open_dataset(path_slope) #map is fixed in time
    dat_slope = dat_slope.assign_coords({'time': pd.to_datetime(fire_slope_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values)})
    dat_slope_daily = dat_slope['slope'].expand_dims({'time': pd.to_datetime(fire_slope_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values)}) #the PWS expanded over all the days

    dat_slope_daily_sub = dat_slope_daily.sel(rows = fire_slope_intersection_xr['row'].values, 
                                              cols = fire_slope_intersection_xr['col'].values,
                                              time = pd.to_datetime(fire_slope_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values), method='nearest')

    print(type(dat_slope_daily_sub))
    ndays = len(fire_slope_intersection_xr[str(day_start_hour)+ 'Z Start Day'])
    
    #preallocate space for the output
    varis = ['day','slope']
    df_slope = generate_df(varis, len(df))
    
    df_slope['day'] = df['12Z Start Day'].values
    df_slope['slope'] = np.nansum(fire_slope_intersection_xr['weights'].values*dat_slope_daily_sub.values, axis=(1,2))
    return df_slope

In [15]:
#do for the coarsened one
def slope_timeseries_coarse(df,day_start_hour):
    #print(df)
    tic = datetime.datetime.now()
    slope_intersections = Parallel(n_jobs=12)(delayed(calculate_intersection)
                                 (df.iloc[ii:ii+1],'BOBCAT_SLOPE_GRID_990M',0.05) 
                                 for ii in range(len(df)))
    toc = datetime.datetime.now()
    print(toc-tic)
    #print([slope_intersections[jj]['weights'].sum() for jj in range(len(slope_intersections))])

    
    fire_slope_intersection=gpd.GeoDataFrame(pd.concat(slope_intersections, ignore_index=True))
    print(len(fire_slope_intersection))
    
    #fire_fuel_fwi_intersection.set_geometry(col='geometry')
    fire_slope_intersection = fire_slope_intersection.drop(columns='geometry')
    fire_slope_intersection = fire_slope_intersection.set_index([str(day_start_hour)+ 'Z Start Day', 'row', 'col'])
    fire_slope_intersection_xr = fire_slope_intersection.to_xarray()
    toc = datetime.datetime.now()
    #print(fire_slope_intersection_xr)

    
    #load in slope data associated with the fire (it's only one dataset)
    
    #open the slope file
    path_slope = '/data2/lthapa/ML_daily/bobcat_slope_990m.nc'
    dat_slope = xr.open_dataset(path_slope) #map is fixed in time
    dat_slope = dat_slope.assign_coords({'time': pd.to_datetime(fire_slope_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values)})
    dat_slope_daily = dat_slope['mean_slope'].expand_dims({'time': pd.to_datetime(fire_slope_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values)}) #the PWS expanded over all the days

    dat_slope_daily_sub = dat_slope_daily.sel(row = fire_slope_intersection_xr['row'].values, 
                                              col = fire_slope_intersection_xr['col'].values,
                                              time = pd.to_datetime(fire_slope_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values), method='nearest')

    print(type(dat_slope_daily_sub))
    ndays = len(fire_slope_intersection_xr[str(day_start_hour)+ 'Z Start Day'])
    
    #preallocate space for the output
    varis = ['day','mean_slope']
    df_slope = generate_df(varis, len(df))
    
    df_slope['day'] = df['12Z Start Day'].values
    df_slope['mean_slope'] = np.nansum(fire_slope_intersection_xr['weights'].values*dat_slope_daily_sub.values, axis=(1,2))
    return df_slope

### Fuel Loading by FWI Class (30m data)

In [ ]:
def fuel_loading_fwi_timeseries(df,day_start_hour):
    loadings = pd.read_csv('FWI_Category_Fuel_Loadings.csv').set_index('VALUE') #we will index by the fuel type!
    """for ii in range(len(df)):
        print(ii)
        today_poly = df.iloc[ii:ii+1] #get the first one, to test it
        fire_fuel_intersection = calculate_loading_grid_intersection(today_poly,50, loadings)
        print(fire_fuel_intersection)"""
        
        
    tic = datetime.datetime.now()    
    fuel_loadings = Parallel(n_jobs=4)(delayed(calculate_loading_grid_intersection)
                                 (df.iloc[ii:ii+1],50, loadings) 
                                 for ii in range(len(df)))
    #df_loadings=pd.DataFrame(fuel_loadings)
    
    toc = datetime.datetime.now()
    print(toc-tic)
    df_loadings = pd.concat(fuel_loadings)
    return df_loadings

In [ ]:
#poly is the polygon for one timestep (in lcc)
#grid is an xarray of the subset of the grid
#grid_names is a string array [0:'lat_center_name',1:'lon_center_name',2:'lat_corner_name',3:'lon_corner_name']

def calculate_loading_grid_intersection(poly,bf, loadings):
    #get the bounds of the buffered polygons
    poly_latlon =poly.to_crs(epsg=5070)

    bounds = poly_latlon.buffer(bf).bounds
    #print(bounds)
    
    left = bounds['minx'].values
    right = bounds['maxx'].values
    bottom=bounds['miny'].values
    top=bounds['maxy'].values
    
    print(left, bottom, top,right)
    
    #select the region of the fuels tiff file that's associated with the polygon
    with rasterio.open('../FCCS_Fuel_Fire_Danger_Metric.tif') as src:
        win = from_bounds(left,bottom,right,top, src.transform)
        #print(win)
        w_from_latlon = src.read(1, window=win)
        w_flat = w_from_latlon.flatten()

    #print(w_from_latlon)
    #build the grid    
    rows = np.arange(win.row_off,win.row_off+win.height-1)
    cols = np.arange(win.col_off,win.col_off+win.width-1)
    COLS,ROWS=np.meshgrid(cols,rows)
    xs, ys = rasterio.transform.xy(src.transform, ROWS, COLS)
    xs=np.array(xs)
    ys=np.array(ys)
    #print(xs.shape)
    
    xs_corner, ys_corner = calculate_grid_cell_corners(xs,ys)
    
    grid = build_grid_xarray(ys_corner, xs_corner, ys, xs)
     
    #add the loadings to the grid
    loading_new_extreme = loadings.loc[w_flat,'Extreme_N'].values.reshape(w_from_latlon.shape)
    loading_new_veryhigh = loadings.loc[w_flat,'VeryHigh_N'].values.reshape(w_from_latlon.shape)
    loading_new_high = loadings.loc[w_flat,'High_N'].values.reshape(w_from_latlon.shape)
    loading_new_moderate = loadings.loc[w_flat,'Moderate_N'].values.reshape(w_from_latlon.shape)
    loading_new_low = loadings.loc[w_flat,'Low_N'].values.reshape(w_from_latlon.shape)
    
    grid['loading_new_extreme'] = xr.Variable(dims = ('rows_ctr','cols_ctr'),data=loading_new_extreme)
    grid['loading_new_veryhigh'] = xr.Variable(dims = ('rows_ctr','cols_ctr'),data=loading_new_veryhigh)
    grid['loading_new_high'] = xr.Variable(dims = ('rows_ctr','cols_ctr'),data=loading_new_high)
    grid['loading_new_moderate'] = xr.Variable(dims = ('rows_ctr','cols_ctr'),data=loading_new_moderate)
    grid['loading_new_low'] = xr.Variable(dims = ('rows_ctr','cols_ctr'),data=loading_new_low)

    #print(grid)
        
    #get the locs, which are the INSIDE of the centers
    df_col = pd.DataFrame({'col_loc':grid['cols_ctr'].values[1:len(grid['cols_ctr'])-2]})
    df_row = pd.DataFrame({'row_loc':grid['rows_ctr'].values[1:len(grid['rows_ctr'])-2]})
    locs = list(product(df_row['row_loc'], df_col['col_loc']))
    print(len(locs))
    
    #make a geodataframe (in paralell of the rows and cols)
    tic = datetime.datetime.now()
    print(tic)
    results = Parallel(n_jobs=8)(delayed(build_one_gridcell)
                                 (grid['LAT_COR'].values, grid['LON_COR'].values,
                                  grid['LAT_CTR'].values, grid['LON_CTR'].values,loc) 
                                 for loc in locs)
    
    #format the grid subset into a dataframs
    df_grid=gpd.GeoDataFrame(results)
    df_grid.columns = ['lat', 'lon', 'row', 'col', 'geometry']
    df_grid.set_geometry(col='geometry',inplace=True,crs='EPSG:5070') #need to say it's in lat/lon before transform to LCC
    df_grid=df_grid.to_crs(epsg=3347)

    
    #intersect the polygon with the grid subset
    njobs = 8
    stride = math.ceil(len(df_grid)/njobs) #round up because we drop duplicates
    intersection = Parallel(n_jobs=njobs)(delayed(gpd.overlay)
                                    (df_grid.iloc[kk:kk+stride], poly, how='intersection',keep_geom_type=False)
                                     for kk in np.arange(0,len(df_grid),stride))
    
    #intersection = gpd.overlay(df_grid,poly,how='intersection',keep_geom_type=False).drop_duplicates()
    toc = datetime.datetime.now()
    print(toc)
    print(toc-tic)
    
    intersection=pd.concat(intersection).drop_duplicates()
    intersection['grid intersection area (ha)'] =intersection['geometry'].area/10000
    intersection['weights'] = intersection['grid intersection area (ha)']/intersection['fire area (ha)'] 
    intersection_xr = intersection.set_index(['row', 'col']).to_xarray()
    
    grid_sub = grid.sel(rows_ctr = np.unique(intersection['row'].values),
                        cols_ctr = np.unique(intersection['col'].values))

    varis = ['day','loading_new_extreme', 'loading_new_veryhigh','loading_new_high', 'loading_new_moderate', 'loading_new_low']
    df_loading = generate_df(varis, len(poly))
    
    df_loading['day'] = poly['12Z Start Day'].values
    for var in varis[1:len(varis)]:
        df_loading[var] = np.nansum(intersection_xr['weights'].values*grid_sub[var].values)
    
    return df_loading

In [ ]:
#makes and saves a geodataframe of a grid given the center and corner points for that grid as 2D matrices
def build_grid_xarray(LAT_COR, LON_COR, LAT_CTR, LON_CTR):
    #loop over the centers
    nrows_center = LAT_CTR.shape[0]
    ncols_center = LAT_CTR.shape[1]
    #print(nrows_center, ncols_center)

    nrows_corner = LAT_COR.shape[0]
    ncols_corner = LAT_COR.shape[1]
    #print(nrows_corner,ncols_corner)
    
    rows_ctr = np.arange(nrows_center)
    cols_ctr = np.arange(ncols_center)
    rows_cor = np.arange(nrows_corner)
    cols_cor = np.arange(ncols_corner)

    
    dat_grid = xr.Dataset(
        data_vars = dict(
            LAT_CTR=(['rows_ctr','cols_ctr'],LAT_CTR),
            LON_CTR=(['rows_ctr','cols_ctr'],LON_CTR),
            LAT_COR=(['rows_cor','cols_cor'],LAT_COR),
            LON_COR=(['rows_cor','cols_cor'],LON_COR),
        ),
        
        coords = dict(
            rows_ctr =(['rows_ctr'],rows_ctr),
            cols_ctr =(['cols_ctr'],cols_ctr),
            rows_cor =(['rows_cor'],rows_cor),
            cols_cor =(['cols_cor'],cols_cor),
        
        )
        
    )
    return(dat_grid)

### Fuel Loading by FWI Class (300m data)

In [34]:
def fuel_loading_fwi_lowres_timeseries(df,day_start_hour):
    print(df)
    tic = datetime.datetime.now()
    fuel_fwi_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (df.iloc[ii:ii+1],'BOBCAT_FUEL_FWI_GRID',0.05) 
                                 for ii in range())
    toc = datetime.datetime.now()
    print(toc-tic)
    print([fuel_fwi_intersections[jj]['weights'].sum() for jj in range(len(fuel_fwi_intersections))])

    
    fire_fuel_fwi_intersection=gpd.GeoDataFrame(pd.concat(fuel_fwi_intersections, ignore_index=True))
    print(len(fire_fuel_fwi_intersection))
    
    #fire_fuel_fwi_intersection.set_geometry(col='geometry')
    fire_fuel_fwi_intersection = fire_fuel_fwi_intersection.drop(columns='geometry')
    fire_fuel_fwi_intersection = fire_fuel_fwi_intersection.set_index([str(day_start_hour)+ 'Z Start Day', 'lat', 'lon'])
    fire_fuel_fwi_intersection_xr = fire_fuel_fwi_intersection.to_xarray()
    toc = datetime.datetime.now()
    print(toc-tic)
    print(fire_fuel_fwi_intersection_xr)

    """
    #load in PWS data associated with the fire (it's only one dataset)
    
    #open the PWS files
    path_fuel_fwi = '/data2/lthapa/bocat_fuel_fwi.nc'
    dat_fuel_fwi = xr.open_dataset(path_fuel_fwi) #map is fixed in time
    
    dat_fuel_fwi = dat_fuel_fwi.assign_coords({'time': pd.to_datetime(fire_pws_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values)})
    
    dat_fuel_fwi_sub = dat_pws_daily.sel(lat = fire_fuel_fwi_intersection_xr['lat'].values, 
                                          lon = fire_fuel_fwi_intersection_xr['lon'].values,
                                          time = pd.to_datetime(fire_pws_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values), method='nearest')
                                          
    ndays = len(fire_pws_intersection_xr[str(day_start_hour)+ 'Z Start Day'])
    
    #preallocate space for the output
    varis = ['day','loading_new_extreme', 'loading_new_veryhigh','loading_new_high', 'loading_new_moderate', 'loading_new_low']
    df_loading = generate_df(varis, len(df))
    
    df_loading['day'] = df['12Z Start Day'].values
    for var in varis[1:len(varis)]:
        df_loading[var] = np.nansum(intersection_xr['weights'].values*grid_sub[var].values)
    return df_pws"""

In [69]:
#poly is the polygon for one timestep (in lcc)
#grid is an xarray of a model grid from the nc file
#grid_names is a string array [0:'lat_center_name',1:'lon_center_name',2:'lat_corner_name',3:'lon_corner_name']

#DOESN'T REALLY WORK THAT WELL
def calculate_intersection_2(poly,dataset_name,bf):
    print(poly['12Z Start Day'])
    #load in the merra grid
    grid = xr.open_dataset(dataset_name+'.nc')

    #get the bounds of the buffered polygons
    poly_latlon =poly.to_crs(epsg=4326)
    
    lat_sub = []
    lon_sub = []
    inds_row =[]
    inds_col = []

    #subset the x,y values by bounds of the multipolygon
    if poly_latlon['geometry'].values[0].geom_type=='Polygon':
        geom_bounds = poly_latlon['geometry'].values[0].buffer(bf).bounds
        inds_row_sel,inds_col_sel = np.where((grid['LON_CTR']>=geom_bounds[0])&(grid['LON_CTR']<=geom_bounds[2])&
                                             (grid['LAT_CTR']>=geom_bounds[1])&(grid['LAT_CTR']<=geom_bounds[3]))
        inds_row.append(inds_row_sel)
        inds_col.append(inds_col_sel)
            
    elif poly_latlon['geometry'].values[0].geom_type=='MultiPolygon':
        for geom in poly_latlon['geometry'].values[0].geoms:
            geom_bounds = geom.buffer(bf).bounds
            inds_row_sel,inds_col_sel = np.where((grid['LON_CTR']>=geom_bounds[0])&(grid['LON_CTR']<=geom_bounds[2])&
                                                 (grid['LAT_CTR']>=geom_bounds[1])&(grid['LAT_CTR']<=geom_bounds[3]))
        inds_row.append(inds_row_sel)
        inds_col.append(inds_col_sel)
        
    inds_row = np.concatenate(inds_row)
    inds_col = np.concatenate(inds_col)

    locs = zip(inds_row,inds_col)
 
    
    #make a geodataframe (in paralell of the rows and cols)
    results = Parallel(n_jobs=8)(delayed(build_one_gridcell)
                                 (grid['LAT_COR'].values, grid['LON_COR'].values,
                                  grid['LAT_CTR'].values, grid['LON_CTR'].values,loc) 
                                 for loc in locs)
    
    #format the grid subset into a dataframs
    df_grid=gpd.GeoDataFrame(results)
    df_grid.columns = ['lat', 'lon', 'row', 'col', 'geometry']
    df_grid.set_geometry(col='geometry',inplace=True,crs='EPSG:4326') #need to say it's in lat/lon before transform to LCC
    df_grid=df_grid.to_crs(epsg=3347)

    
    #intersect the polygon with the grid subset
    intersection = gpd.overlay(df_grid, poly, how='intersection',keep_geom_type=False).drop_duplicates()
    intersection['grid intersection area (ha)'] =intersection['geometry'].area/10000
    intersection['weights'] = intersection['grid intersection area (ha)']/intersection['fire area (ha)'] 
    
    return intersection


### MERRA

In [ ]:
def merra_timeseries(df,day_start_hour):
    df_merra = pd.DataFrame({'day': np.zeros(len(df)),'temp':np.zeros(len(df)), 'vpd':np.zeros(len(df)), 
                             'wind':np.zeros(len(df)),'hd0w0':np.zeros(len(df)), 'hd1w0':np.zeros(len(df)),
                             'hd2w0':np.zeros(len(df)),'hd3w0':np.zeros(len(df)), 'hd4w0':np.zeros(len(df)),
                             'hd5w0':np.zeros(len(df))}) 
    
    #do the intersection, in parallel
    merra_intersections = Parallel(n_jobs=6)(delayed(calculate_intersection)
                                 (df.iloc[ii:ii+1],'MERRA_GRID',0.5) 
                                 for ii in range(len(df)))
    fire_merra_intersection=gpd.GeoDataFrame(pd.concat(merra_intersections, ignore_index=True))
    fire_merra_intersection.set_geometry(col='geometry')
    
    
    #loop over all of the days we have intersections
    times_intersect = np.unique(fire_merra_intersection[str(day_start_hour)+ 'Z Start Day'].values)
    times_utc = np.unique(fire_merra_intersection['UTC Day'].values)
    
    count = 0
    for today in times_intersect:
        #get the time
        df_sub = fire_merra_intersection.iloc[np.where(fire_merra_intersection[str(day_start_hour)+ 'Z Start Day'].values==today)]
        df_sub = df_sub.set_index([str(day_start_hour)+ 'Z Start Day', 'lat', 'lon'])
        intersection_sub = df_sub.to_xarray() #polygon and weights for today
        print(intersection_sub['weights'].sum(dim=['lat','lon']))
        times_back = pd.date_range(start=np.datetime64(today)-np.timedelta64(5,'D'), end=np.datetime64(today)+np.timedelta64(1,'D'))
        files_back = make_merra_file_namelist(times_back)
        
        #load in all the merra files associated with this lookback window
        dat_merra = xr.open_mfdataset(files_back,concat_dim='time',combine='nested',compat='override', coords='all')
    
        #add the derived data (svp, vp, vpd)
        dat_merra=dat_merra.assign(ESAT=sat_vap_press(dat_merra.TLML))
        dat_merra=dat_merra.assign(E=vap_press(dat_merra.QLML, dat_merra.TLML))
        dat_merra=dat_merra.assign(VPD=dat_merra.ESAT-dat_merra.E)
        
        merra_daily_mean = dat_merra.resample(time='24H',base=day_start_hour, label='left').mean(dim='time') #take the daily mean        
        merra_daily_mean_region = merra_daily_mean.sel(lat = np.unique(intersection_sub['lat'].values),
                                  lon = np.unique(intersection_sub['lon'].values)) #get the location of the overlaps
        
        hd0 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        hd1 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(1,'D')).values)*
                     (intersection_sub['weights'].values))
        hd2 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(2,'D')).values)*
                     (intersection_sub['weights'].values))
        hd3 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(3,'D')).values)*
                     (intersection_sub['weights'].values))
        hd4 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(4,'D')).values)*
                     (intersection_sub['weights'].values))
        hd5 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(5,'D')).values)*
                     (intersection_sub['weights'].values))
        w = np.nansum((merra_daily_mean_region['SPEEDLML'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        t = np.nansum((merra_daily_mean_region['TLML'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        
        df_merra.iloc[count,:] = [today+ ' '+str(day_start_hour)+':00:00',t,hd0,w,hd0*w,hd1*w,hd2*w,hd3*w,hd4*w,hd5*w]
        dat_merra.close()
        count =count+1
    return df_merra
    

In [ ]:
def make_merra_file_namelist(time):
    base_filename = '/data2/lthapa/YEAR/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.FULLDATE.nc4'
    base_filename_list = np.repeat(base_filename, len(time))

    
    for jj in range(len(time)):
        base_filename_list[jj] = base_filename_list[jj].replace('YEAR',time[jj].strftime('%Y')).\
                                    replace('FULLDATE',time[jj].strftime('%Y%m%d'))
        if (time[jj].strftime('%Y%m')=='202009'):
            base_filename_list[jj] = base_filename_list[jj].replace('400','401')
    return base_filename_list

### HRRR

In [57]:
def hrrr_timeseries_chi(df, day_start_hour):
    varis_derived = ['day','temp_850','temp_700','dewpt_850','ca','cb','chi']
    df_hrrr_derived = generate_df(varis_derived, len(df))
    
    #do the intersection, in parallel
    tic=datetime.datetime.now()
    #print(tic)
    hrrr_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (df.iloc[ii:ii+1],'HRRR_GRID',0.05) 
                                 for ii in range(len(df)))
    fire_hrrr_intersection=gpd.GeoDataFrame(pd.concat(hrrr_intersections, ignore_index=True))
    fire_hrrr_intersection.set_geometry(col='geometry')
    fire_hrrr_intersection = fire_hrrr_intersection.set_index([str(day_start_hour)+ 'Z Start Day', 'row', 'col'])
    fire_hrrr_intersection=fire_hrrr_intersection[~fire_hrrr_intersection.index.duplicated()]
    toc = datetime.datetime.now()
    print(toc-tic)
    
    fire_hrrr_intersection_xr = fire_hrrr_intersection.to_xarray()
    
    times = pd.date_range(np.datetime64(df[str(day_start_hour)+ 'Z Start Day'].iloc[0]),
                        np.datetime64(df[str(day_start_hour)+ 'Z Start Day'].iloc[len(df)-1])+
                        np.timedelta64(1,'D'), freq='H')
    chi_filenames,times_back_used = make_file_namelist(times,'/data2/lthapa/ML_daily/pygraf/Processed_HRRR_CHI_YYYYMMDDHH.nc')
    
    dat_hrrr = xr.open_mfdataset(chi_filenames, concat_dim='Time', combine='nested',compat='override',coords='all')
    dat_hrrr = dat_hrrr.assign_coords({'Time':times_back_used}) 
    hrrr_daily_mean = dat_hrrr.resample(Time='24H',base=day_start_hour, label='left').mean(dim='Time') #take the daily mean
    
    print(hrrr_daily_mean['Time'].values)
    times_sel = [fire_hrrr_intersection_xr[str(day_start_hour)+'Z Start Day'].values[jj]+' '+str(day_start_hour)+':00:00' for jj in range(len(fire_hrrr_intersection_xr[str(day_start_hour)+'Z Start Day'].values))]
    print(times_sel)
    #select the locations and times we want
    dat_hrrr_sub = hrrr_daily_mean.isel(grid_yt = fire_hrrr_intersection_xr['row'].values.astype(int), 
                                         grid_xt = fire_hrrr_intersection_xr['col'].values.astype(int)).sel(
                        Time = pd.to_datetime(times_sel))
    
    df_hrrr_derived['day'].iloc[:] = pd.to_datetime(fire_hrrr_intersection_xr[str(day_start_hour)+'Z Start Day'].values)
    for var in varis_derived[1:]:
        df_hrrr_derived[var] = np.nansum(fire_hrrr_intersection_xr['weights'].values*dat_hrrr_sub[var].values, axis=(1,2))
        

    return df_hrrr_derived

In [ ]:
def hrrr_timeseries(df,day_start_hour):  
    df_hrrr_derived = pd.DataFrame({'day': np.zeros(len(df)),'hd0w0':np.zeros(len(df)), 'hd1w0':np.zeros(len(df)),
                             'hd2w0':np.zeros(len(df)),'hd3w0':np.zeros(len(df)), 'hd4w0':np.zeros(len(df)),
                             'hd5w0':np.zeros(len(df))})
    
    df_hrrr_raw = pd.DataFrame({'temp_2m':np.zeros(len(df)), 'q_2m':np.zeros(len(df)), 
                             'gust_sfc':np.zeros(len(df)),'veggie':np.zeros(len(df)), 'dewpt':np.zeros(len(df)),
                             'weasd':np.zeros(len(df)),'soilm':np.zeros(len(df)), 'esat_2m':np.zeros(len(df)),
                             'e_2m': np.zeros(len(df)),'vpd_2m':np.zeros(len(df)), 'hwp':np.zeros(len(df)), 
                             'gust_sfc':np.zeros(len(df)),'veggie':np.zeros(len(df)), 'dewpt':np.zeros(len(df)),
                             'veg_term':np.zeros(len(df)),'gust_max_term':np.zeros(len(df)), 'dd_term':np.zeros(len(df)),
                             'mois_term': np.zeros(len(df)),'temp_2m':np.zeros(len(df)), 'q_2m':np.zeros(len(df)), 
                             'gust_sfc':np.zeros(len(df)),'snowc_term':np.zeros(len(df))})
    
    varis = ['temp_2m', 'q_2m','gust_sfc','veggie', 'dewpt','weasd','soilm', 'esat_2m','e_2m',
             'vpd_2m', 'hwp','gust_sfc','veggie', 'dewpt','veg_term','gust_max_term', 'dd_term',
            'mois_term','temp_2m', 'q_2m','gust_sfc','snowc_term']
    
    #do the intersection, in parallel
    tic=time.time()
    hrrr_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (fire_daily.iloc[ii:ii+1],'HRRR_GRID',0.05) 
                                 for ii in range(len(fire_daily)))
    fire_hrrr_intersection=gpd.GeoDataFrame(pd.concat(hrrr_intersections, ignore_index=True))
    fire_hrrr_intersection.set_geometry(col='geometry')
    toc = time.time()
    print(toc-tic)
    
    
    #loop over all of the days we have intersections
    times_intersect = np.unique(fire_hrrr_intersection[str(day_start_hour)+ 'Z Start Day'].values)
    times_utc = np.unique(fire_hrrr_intersection['UTC Day'].values)
    
    count = 0
    for today in times_intersect:
        print(today)
        #get the time
        df_sub = fire_hrrr_intersection.iloc[np.where(fire_hrrr_intersection[str(day_start_hour)+ 'Z Start Day'].values==today)]
        df_sub = df_sub.set_index([str(day_start_hour)+ 'Z Start Day', 'row', 'col'])
        df_sub=df_sub[~df_sub.index.duplicated()]
        intersection_sub = df_sub.to_xarray() #polygon and weights for today
        
        times_back = pd.date_range(start=np.datetime64(today)-np.timedelta64(5,'D'), end=np.datetime64(today)+
                                   np.timedelta64(1,'D'),freq='H')
        files_back,times_back_used = make_file_namelist(times_back,'/data2/lthapa/ML_daily/pygraf/Processed_HRRR_YYYYMMDDHH.nc')

        #load in all the merra files associated with this lookback window
        dat_hrrr = xr.open_mfdataset(files_back,concat_dim='time',combine='nested',compat='override', coords='all')
        dat_hrrr = dat_hrrr.assign_coords({'time': times_back_used})

        #add the derived data (svp, vp, vpd)
        dat_hrrr=dat_hrrr.assign(esat_2m=sat_vap_press(dat_hrrr.temp_2m))
        dat_hrrr=dat_hrrr.assign(e_2m=vap_press(dat_hrrr.q_2m, dat_hrrr.temp_2m))
        dat_hrrr=dat_hrrr.assign(VPD=dat_hrrr.esat_2m-dat_hrrr.e_2m)
        
        hrrr_daily_mean = dat_hrrr.resample(time='24H',base=day_start_hour, label='left').mean(dim='time') #take the daily mean        
        
        hrrr_daily_mean_region = hrrr_daily_mean.sel(grid_yt = np.unique(intersection_sub['row'].values),
                                                    grid_xt = np.unique(intersection_sub['col'].values)) #get the location of the overlaps
        print(hrrr_daily_mean_region['time'].values)
        hd0 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        hd1 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(1,'D')).values)*
                     (intersection_sub['weights'].values))
        hd2 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(2,'D')).values)*
                     (intersection_sub['weights'].values))
        hd3 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(3,'D')).values)*
                     (intersection_sub['weights'].values))
        hd4 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(4,'D')).values)*
                     (intersection_sub['weights'].values))
        hd5 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(5,'D')).values)*
                     (intersection_sub['weights'].values))
        w = np.nansum((hrrr_daily_mean_region['gust_sfc'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        t = np.nansum((hrrr_daily_mean_region['temp_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        
        df_hrrr_derived.iloc[count,:] = [today+ ' '+str(day_start_hour)+':00:00',hd0*w,hd1*w,hd2*w,hd3*w,hd4*w,hd5*w]
        
        for var in varis:
            df_hrrr_raw[var].iloc[count] = np.nansum(intersection_sub['weights'].values*hrrr_daily_mean_region[var].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values, axis=(1,2))
        
        df_hrrr = pd.concat([df_hrrr_derived,df_hrrr_raw],axis=1)
        dat_hrrr.close()
        count =count+1
    return df_hrrr

In [ ]:
def hrrr_timeseries_ws(df,day_start_hour):  #with the wind speed
    df_hrrr_derived = pd.DataFrame({'day': np.zeros(len(df)),'hd0w0':np.zeros(len(df)), 'hd1w0':np.zeros(len(df)),
                             'hd2w0':np.zeros(len(df)),'hd3w0':np.zeros(len(df)), 'hd4w0':np.zeros(len(df)),
                             'hd5w0':np.zeros(len(df))})
    
    df_hrrr_raw = pd.DataFrame({'temp_2m':np.zeros(len(df)), 'q_2m':np.zeros(len(df)), 
                             'wind_speed':np.zeros(len(df)),'veggie':np.zeros(len(df)), 'dewpt':np.zeros(len(df)),
                             'weasd':np.zeros(len(df)),'soilm':np.zeros(len(df)), 'esat_2m':np.zeros(len(df)),
                             'e_2m': np.zeros(len(df)),'vpd_2m':np.zeros(len(df)), 'hwp':np.zeros(len(df)), 
                             'veggie':np.zeros(len(df)), 'dewpt':np.zeros(len(df)),
                             'veg_term':np.zeros(len(df)),'gust_max_term':np.zeros(len(df)), 'dd_term':np.zeros(len(df)),
                             'mois_term': np.zeros(len(df)),'temp_2m':np.zeros(len(df)), 'q_2m':np.zeros(len(df)), 
                             'snowc_term':np.zeros(len(df))})
    
    varis = ['temp_2m', 'q_2m','wind_speed','veggie', 'dewpt','weasd','soilm', 'esat_2m','e_2m',
             'vpd_2m', 'hwp','veggie', 'dewpt','veg_term','gust_max_term', 'dd_term',
            'mois_term','temp_2m', 'q_2m','snowc_term']
    
    #do the intersection, in parallel
    tic=time.time()
    #print(tic)
    hrrr_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (df.iloc[ii:ii+1],'HRRR_GRID',0.05) 
                                 for ii in range(len(df)))
    fire_hrrr_intersection=gpd.GeoDataFrame(pd.concat(hrrr_intersections, ignore_index=True))
    fire_hrrr_intersection.set_geometry(col='geometry')
    toc = time.time()
    print(toc-tic)
    
    
    #loop over all of the days we have intersections
    times_intersect = np.unique(fire_hrrr_intersection[str(day_start_hour)+ 'Z Start Day'].values)
    times_utc = np.unique(fire_hrrr_intersection['UTC Day'].values)
    
    count = 0
    for today in times_intersect:
        print(today)
        #get the time
        df_sub = fire_hrrr_intersection.iloc[np.where(fire_hrrr_intersection[str(day_start_hour)+ 'Z Start Day'].values==today)]
        df_sub = df_sub.set_index([str(day_start_hour)+ 'Z Start Day', 'row', 'col'])
        df_sub=df_sub[~df_sub.index.duplicated()]
        intersection_sub = df_sub.to_xarray() #polygon and weights for today
        
        times_back = pd.date_range(start=np.datetime64(today)-np.timedelta64(5,'D'), end=np.datetime64(today)+
                                   np.timedelta64(1,'D'),freq='H')
        files_back,times_back_used = make_file_namelist(times_back,'/data2/lthapa/ML_daily/pygraf/Processed_HRRR_YYYYMMDDHH.nc')

        #load in all the merra files associated with this lookback window
        dat_hrrr = xr.open_mfdataset(files_back,concat_dim='time',combine='nested',compat='override', coords='all')
        dat_hrrr = dat_hrrr.assign_coords({'time': times_back_used})

        #add the derived data (svp, vp, vpd)
        dat_hrrr=dat_hrrr.assign(esat_2m=sat_vap_press(dat_hrrr.temp_2m))
        dat_hrrr=dat_hrrr.assign(e_2m=vap_press(dat_hrrr.q_2m, dat_hrrr.temp_2m))
        dat_hrrr=dat_hrrr.assign(VPD=dat_hrrr.esat_2m-dat_hrrr.e_2m)
        
        hrrr_daily_mean = dat_hrrr.resample(time='24H',base=day_start_hour, label='left').mean(dim='time') #take the daily mean        
        
        hrrr_daily_mean_region = hrrr_daily_mean.sel(grid_yt = np.unique(intersection_sub['row'].values),
                                                    grid_xt = np.unique(intersection_sub['col'].values)) #get the location of the overlaps
        #print(hrrr_daily_mean_region['time'].values)
        hd0 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        hd1 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(1,'D')).values)*
                     (intersection_sub['weights'].values))
        hd2 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(2,'D')).values)*
                     (intersection_sub['weights'].values))
        hd3 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(3,'D')).values)*
                     (intersection_sub['weights'].values))
        hd4 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(4,'D')).values)*
                     (intersection_sub['weights'].values))
        hd5 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(5,'D')).values)*
                     (intersection_sub['weights'].values))
        w = np.nansum((hrrr_daily_mean_region['wind_speed'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        t = np.nansum((hrrr_daily_mean_region['temp_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        
        df_hrrr_derived.iloc[count,:] = [today+ ' '+str(day_start_hour)+':00:00',hd0*w,hd1*w,hd2*w,hd3*w,hd4*w,hd5*w]
        
        for var in varis:
            df_hrrr_raw[var].iloc[count] = np.nansum(intersection_sub['weights'].values*hrrr_daily_mean_region[var].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values, axis=(1,2))
        
        df_hrrr = pd.concat([df_hrrr_derived,df_hrrr_raw],axis=1)
        dat_hrrr.close()
        count =count+1
        #print(df_hrrr)
    return df_hrrr

In [ ]:
def hrrr_timeseries_sub_daily(df,day_start_hour, time_ave_interval):  #with the wind speed
    #set up the output dfs
    varis_raw = ['temp_2m', 'q_2m','wind_speed','veggie', 'dewpt','weasd','soilm', 'esat_2m','e_2m','vpd_2m', 'hwp','veggie', 'dewpt','veg_term','gust_max_term', 'dd_term','mois_term','temp_2m', 'q_2m','snowc_term']
    df_hrrr_raw = generate_df(varis_raw, len(df))

    varis_derived = ['day','hd0w0','hd05w0','hd1w0','hd15w0','hd2w0','hd25w0','hd3w0']
    df_hrrr_derived = generate_df(varis_derived, len(df))

    
    #do the intersection, in parallel
    #tic=datetime.time.time()
    #tic=datetime.time.time()
    #print(tic)
    hrrr_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (df.iloc[ii:ii+1],'HRRR_GRID',0.05) 
                                 for ii in range(len(df)))
    fire_hrrr_intersection=gpd.GeoDataFrame(pd.concat(hrrr_intersections, ignore_index=True))
    fire_hrrr_intersection.set_geometry(col='geometry')
    #toc = time.time()
    #print(toc-tic)
    
    
    #loop over all of the days we have intersections
    times_intersect = np.unique(fire_hrrr_intersection['time'].values)
    times_utc = np.unique(fire_hrrr_intersection['UTC Day'].values)
    print(times_intersect)
    count = 0
    for today in times_intersect:
        print(today)
        print(pd.to_datetime(today).time())
        #print(today[0])
        
        #get the time
        df_sub = fire_hrrr_intersection.iloc[np.where(fire_hrrr_intersection[str(day_start_hour)+ 'Z Start Day'].values==today)]
        df_sub = df_sub.set_index([str(day_start_hour)+ 'Z Start Day', 'row', 'col'])
        df_sub=df_sub[~df_sub.index.duplicated()]
        intersection_sub = df_sub.to_xarray() #polygon and weights for today
        
        times_back = pd.date_range(start=np.datetime64(today)-np.timedelta64(5,'D'), end=np.datetime64(today)+
                                   np.timedelta64(1,'D'),freq='H')
        files_back,times_back_used = make_file_namelist(times_back,'/data2/lthapa/ML_daily/pygraf/Processed_HRRR_YYYYMMDDHH.nc')

        #load in all the merra files associated with this lookback window
        dat_hrrr = xr.open_mfdataset(files_back,concat_dim='time',combine='nested',compat='override', coords='all')
        dat_hrrr = dat_hrrr.assign_coords({'time': times_back_used})

        #add the derived data (svp, vp, vpd)
        dat_hrrr=dat_hrrr.assign(esat_2m=sat_vap_press(dat_hrrr.temp_2m))
        dat_hrrr=dat_hrrr.assign(e_2m=vap_press(dat_hrrr.q_2m, dat_hrrr.temp_2m))
        dat_hrrr=dat_hrrr.assign(VPD=dat_hrrr.esat_2m-dat_hrrr.e_2m)
        
        hrrr_daily_mean = dat_hrrr.resample(time=str(time_ave_interval)+'H', label='left').mean(dim='time') #take the daily mean        
        hrrr_daily_mean = hrrr_daily_mean.sel(time=pd.to_datetime(today).time())
        hrrr_daily_mean_region = hrrr_daily_mean.sel(grid_yt = np.unique(intersection_sub['row'].values),
                                                    grid_xt = np.unique(intersection_sub['col'].values)
                                                    ) #get the location of the overlaps
        
        print(hrrr_daily_mean_region['time'])
        
        break
        """

In [ ]:
def hrrr_directional_hdw(df_split, df_poly,df_perim,day_start_hour, time_ave_interval):  #with the wind speed
    #set up the output dfs
    
    varis_derived = ['day','hd0w0','hd05w0','hd1w0','hd15w0','hd2w0','hd25w0','hd3w0',
                    'hd0w0_d','hd05w0_d','hd1w0_d','hd15w0_d','hd2w0_d','hd25w0_d','hd3w0_d']
    df_hrrr_derived = generate_df(varis_derived, len(df_split))
    
    varis_raw = ['temp_2m', 'q_2m','wind_speed','veggie', 'dewpt','weasd','soilm', 'esat_2m','e_2m',
             'vpd_2m', 'hwp','veggie', 'dewpt','veg_term','gust_max_term', 'dd_term',
            'mois_term','temp_2m', 'q_2m','snowc_term']
    df_hrrr_raw = generate_df(varis_raw, len(df_split))
    
    #do the intersection, in parallel
    hrrr_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (df_poly.iloc[ii:ii+1],'HRRR_GRID',0.05) 
                                 for ii in range(len(df_poly)))
    fire_hrrr_intersection=gpd.GeoDataFrame(pd.concat(hrrr_intersections, ignore_index=True))
    fire_hrrr_intersection.set_geometry(col='geometry')

    #loop over all of the days we have intersections
    times_intersect = np.unique(df_poly['time'].values) #every half daily time step where the polygons intersect the grid

    count=0
    for today in times_intersect:
        print(today)
        
        #get the intersection information for today
        df_intersect = fire_hrrr_intersection.iloc[np.where(fire_hrrr_intersection['time'].values==today)]
        df_intersect = df_intersect.set_index([str(day_start_hour)+ 'Z Start Day', 'row', 'col'])
        df_intersect=df_intersect[~df_intersect.index.duplicated()] #get rid of duplicates
        intersection_sub = df_intersect.to_xarray() #polygon and weights for today
        #print(intersection_sub)
        
        #define the lookback window
        times_back = pd.date_range(start=np.datetime64(today)-np.timedelta64(3,'D'), end=np.datetime64(today)+
                                   np.timedelta64(1,'D'),freq='H')
        files_back,times_back_used = make_file_namelist(times_back,'/data2/lthapa/ML_daily/pygraf/Processed_HRRR_YYYYMMDDHH.nc')

        
        #load in all the merra files associated with this lookback window
        dat_hrrr = xr.open_mfdataset(files_back,concat_dim='time',combine='nested',compat='override', coords='all')
        dat_hrrr = dat_hrrr.assign_coords({'time': times_back_used})

        #take the average, 12 hourly, based on time dimension
        hrrr_subdaily_mean = dat_hrrr.resample(time=str(time_ave_interval)+'H', label='left').mean(dim='time') #take the daily mean        
        #print(hrrr_subdaily_mean)
        hrrr_subdaily_mean_region = hrrr_subdaily_mean.sel(grid_yt = np.unique(intersection_sub['row'].values),
                                                    grid_xt = np.unique(intersection_sub['col'].values)
                                                    ) #get the location of the overlaps
        
        

        """#GET THE POLYGON VARIABLES
        for var_raw in varis_raw:
            df_hrrr_raw[var_raw].iloc[count] = \
                    np.nansum(intersection_sub['weights'].values*hrrr_subdaily_mean_region[var_raw].sel(time=np.datetime64(today)).values, axis=(1,2))
        
        
        hd00 = np.nansum((hrrr_subdaily_mean_region['vpd_2m'].sel(time=np.datetime64(today)).values)*
                        (intersection_sub['weights'].values))
        hd05 = np.nansum((hrrr_subdaily_mean_region['vpd_2m'].sel(time=np.datetime64(today)-np.timedelta64(12,'h')).values)*
                     (intersection_sub['weights'].values))
        hd10 = np.nansum((hrrr_subdaily_mean_region['vpd_2m'].sel(time=np.datetime64(today)-np.timedelta64(24,'h')).values)*
                     (intersection_sub['weights'].values))
        hd15 = np.nansum((hrrr_subdaily_mean_region['vpd_2m'].sel(time=np.datetime64(today)-np.timedelta64(36,'h')).values)*
                     (intersection_sub['weights'].values))
        hd20 = np.nansum((hrrr_subdaily_mean_region['vpd_2m'].sel(time=np.datetime64(today)-np.timedelta64(48,'h')).values)*
                     (intersection_sub['weights'].values))
        hd25 = np.nansum((hrrr_subdaily_mean_region['vpd_2m'].sel(time=np.datetime64(today)-np.timedelta64(60,'h')).values)*
                     (intersection_sub['weights'].values))
        hd30 = np.nansum((hrrr_subdaily_mean_region['vpd_2m'].sel(time=np.datetime64(today)-np.timedelta64(72,'h')).values)*
                     (intersection_sub['weights'].values))
        w = np.nansum((hrrr_subdaily_mean_region['wind_speed'].sel(time=np.datetime64(today)).values)*
                        (intersection_sub['weights'].values))           
    
        
        """
        #GET THE PERIMETER VARIABLES (DIRECTIONAL HDW)
        split = df_split[df_split['time']==today]['fireline_split'] #get the firelines
        #print(split)
        
        segs_list = list(split.iloc[0].geoms)
        segs = gpd.GeoDataFrame({'ind':np.arange(0,len(segs_list)),
                        'time': np.repeat(today,len(segs_list)),
                        'geometry':segs_list})
        segs = segs[segs.length>0.0005]
        #print(segs)
        
        perimeter = df_perim['perimeter'][df_poly['time']==today]
        perimeter_bounds = perimeter.buffer(0.025).bounds
        
        perp_segs = find_perp_outside_segments(segs,0.005, perimeter.values) #get the perpindicular segments
        
            
        #subset the wind data
        time_sel,grid_yt_sel, grid_xt_sel = np.where((hrrr_subdaily_mean['grid_lont']<perimeter_bounds['maxx'].values)&
                                    (hrrr_subdaily_mean['grid_lont']>perimeter_bounds['minx'].values)&
                                    (hrrr_subdaily_mean['grid_latt']<perimeter_bounds['maxy'].values)&
                                    (hrrr_subdaily_mean['grid_latt']>perimeter_bounds['miny'].values))

        hrrr_subdaily_mean_wind_region = hrrr_subdaily_mean.sel(grid_yt=np.unique(grid_yt_sel),grid_xt=np.unique(grid_xt_sel))
        hrrr_subdaily_mean_wind_region= hrrr_subdaily_mean_wind_region.set_coords(('grid_lont','grid_latt'))
        print(hrrr_subdaily_mean_wind_region)
            
        """f_var = si.interp2d(hrrr_subdaily_mean_wind_region['grid_lont'].values.flatten(), 
                            hrrr_subdaily_mean_wind_region['grid_latt'].values.flatten(), 
                            hrrr_subdaily_mean_wind_region['uwind_10m'].values.flatten(),kind='linear')
        var_new = np.array([f_var(perp_segs['x_mid'].values[kk], perp_segs['y_mid'].values[kk])
                            for kk in range(len(perp_segs))])
        print(hrrr_subdaily_mean_wind_region['grid_lont'].values)
        print(hrrr_subdaily_mean_wind_region['grid_latt'].values)
        print(hrrr_subdaily_mean_wind_region['uwind_10m'].values)
        print(var_new)"""
        print(interp_at_perps(hrrr_subdaily_mean_wind_region.sel(time=np.datetime64(today)),perp_segs, 'uwind_10m'))
        print(interp_at_perps(hrrr_subdaily_mean_wind_region.sel(time=np.datetime64(today)),perp_segs, 'vpd_2m'))
        print(interp_at_perps(hrrr_subdaily_mean_wind_region.sel(time=np.datetime64(today)-np.timedelta64(12,'h')),
                              perp_segs, 'vpd_2m'))
        print(interp_at_perps(hrrr_subdaily_mean_wind_region.sel(time=np.datetime64(today)-np.timedelta64(24,'h')),
                              perp_segs, 'vpd_2m'))
        print(interp_at_perps(hrrr_subdaily_mean_wind_region.sel(time=np.datetime64(today)-np.timedelta64(36,'h')),
                              perp_segs, 'vpd_2m'))
        print(interp_at_perps(hrrr_subdaily_mean_wind_region.sel(time=np.datetime64(today)-np.timedelta64(48,'h')),
                              perp_segs, 'vpd_2m'))
        print(interp_at_perps(hrrr_subdaily_mean_wind_region.sel(time=np.datetime64(today)-np.timedelta64(60,'h')),
                              perp_segs, 'vpd_2m'))
        print(interp_at_perps(hrrr_subdaily_mean_wind_region.sel(time=np.datetime64(today)-np.timedelta64(72,'h')),
                              perp_segs, 'vpd_2m'))
            


                        
        
        """#calculate HDW point by point, then take the length weighted average
        perp_wind_speed=calculate_perp_ws(hrrr_subdaily_mean_wind_region.sel(time=today),perp_segs)
        hd00_d = interp_at_perps(hrrr_subdaily_mean_wind_region.sel(time=np.datetime64(today)),perp_segs, 'vpd_2m')
        hd05_d = interp_at_perps(hrrr_subdaily_mean_wind_region.sel(time=np.datetime64(today)-np.timedelta64(12,'h')),perp_segs, 'vpd_2m')
        hd10_d = interp_at_perps(hrrr_subdaily_mean_wind_region.sel(time=np.datetime64(today)-np.timedelta64(24,'h')),perp_segs, 'vpd_2m')
        hd15_d = interp_at_perps(hrrr_subdaily_mean_wind_region.sel(time=np.datetime64(today)-np.timedelta64(36,'h')),perp_segs, 'vpd_2m')
        hd20_d = interp_at_perps(hrrr_subdaily_mean_wind_region.sel(time=np.datetime64(today)-np.timedelta64(48,'h')),perp_segs, 'vpd_2m')
        hd25_d = interp_at_perps(hrrr_subdaily_mean_wind_region.sel(time=np.datetime64(today)-np.timedelta64(60,'h')),perp_segs, 'vpd_2m')
        hd30_d = interp_at_perps(hrrr_subdaily_mean_wind_region.sel(time=np.datetime64(today)-np.timedelta64(72,'h')),perp_segs, 'vpd_2m')
        
        #print(perp_wind_speed,hd00_d)
        # do the length weighting
        fireline_length = segs['geometry'].length
        fireline_length_fraction = fireline_length/(np.nansum(fireline_length))
        
        #save the outputs
        df_hrrr_derived.iloc[count,:] = [today,hd00*w,hd05*w,hd10*w,hd15*w,hd20*w,hd25*w,hd30*w,
                                        np.nansum(fireline_length_fraction*perp_wind_speed*hd00_d),
                                        np.nansum(fireline_length_fraction*perp_wind_speed*hd05_d),
                                        np.nansum(fireline_length_fraction*perp_wind_speed*hd10_d),
                                        np.nansum(fireline_length_fraction*perp_wind_speed*hd15_d),
                                        np.nansum(fireline_length_fraction*perp_wind_speed*hd20_d),
                                        np.nansum(fireline_length_fraction*perp_wind_speed*hd25_d),
                                        np.nansum(fireline_length_fraction*perp_wind_speed*hd30_d)
                                        ]
        print(df_hrrr_derived)"""
        
        """
        weighted_perp_wind = (perp_segs['fireline_length_fraction'].values*perp_wind_speed).sum() #this should be the length-weighted wind speed
    
        
        # get the interpolated, length weithged VPD

       """
        
        #break #to only do the first day
        count = count+1
        

In [ ]:
def hrrr_directional_hdw_test(df_split, df_poly,df_perim,day_start_hour, time_ave_interval):  #with the wind speed
    #set up the output dfs
    
    varis_derived = ['day','hd0w0','hd0w0_edge','hd0w0_edge_lw', 'hd0w0_perp',
                     'hd0w0_perp_lw', 'hd0w0_perp_out', 'hd0w0_perp_out_lw']
    df_hrrr_derived = generate_df(varis_derived, len(df_split))
    
    varis_raw = ['vpd_2m','wind_speed']
    df_hrrr_raw = generate_df(varis_raw, len(df_split))
    
    #do the intersection, in parallel
    hrrr_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (df_poly.iloc[ii:ii+1],'HRRR_GRID',0.05) 
                                 for ii in range(len(df_poly)))
    fire_hrrr_intersection=gpd.GeoDataFrame(pd.concat(hrrr_intersections, ignore_index=True))
    fire_hrrr_intersection.set_geometry(col='geometry')
    print(fire_hrrr_intersection)
    #loop over all of the days we have intersections
    times_intersect = np.unique(df_poly['time'].values) #every half daily time step where the polygons intersect the grid
    print(times_intersect)
    count=0
    for today in times_intersect:
        print(today)
        #get the intersection information for today
        df_intersect = fire_hrrr_intersection.iloc[np.where(fire_hrrr_intersection['time'].values==today)]
        df_intersect = df_intersect.set_index([str(day_start_hour)+ 'Z Start Day', 'row', 'col'])
        df_intersect=df_intersect[~df_intersect.index.duplicated()] #get rid of duplicates
        intersection_sub = df_intersect.to_xarray() #polygon and weights for today
        
        #define the lookback window
        times_back = pd.date_range(start=np.datetime64(today)-np.timedelta64(1,'D'), end=np.datetime64(today)+
                                   np.timedelta64(1,'D'),freq='H')
        files_back,times_back_used = make_file_namelist(times_back,'/data2/lthapa/ML_daily/pygraf/Processed_HRRR_YYYYMMDDHH.nc')

        
        #load in all the merra files associated with this lookback window
        dat_hrrr = xr.open_mfdataset(files_back,concat_dim='time',combine='nested',compat='override', coords='all')
        dat_hrrr = dat_hrrr.assign_coords({'time': times_back_used})        
        
        
        #GET THE PERIMETER VARIABLES
        split = df_split[df_split['time']==today]['fireline_split'] #get the firelines        
        segs_list = list(split.iloc[0].geoms)
        segs = gpd.GeoDataFrame({'ind':np.arange(0,len(segs_list)),
                        'time': np.repeat(today,len(segs_list)),
                        'geometry':segs_list})
        segs = segs[segs.length>0.0005]
        #print(segs)
        perimeter = df_perim['perimeter'][df_poly['time']==today]
        perimeter_bounds = perimeter.buffer(0.05).bounds
        #print(perimeter_bounds)
        perp_segs = find_perp_outside_segments(segs,0.005, perimeter.values) #get the perpindicular segments
        
        #take the average, 12 hourly, based on time dimension
        hrrr_subdaily_mean = dat_hrrr.resample(time=str(time_ave_interval)+'H', label='left').mean(dim='time') #take the daily mean         
        hrrr_00 = hrrr_subdaily_mean.sel(time=today)
        
        grid_yt_sel, grid_xt_sel = np.where((hrrr_00['grid_lont']<perimeter_bounds['maxx'].values)&
                                    (hrrr_00['grid_lont']>perimeter_bounds['minx'].values)&
                                    (hrrr_00['grid_latt']<perimeter_bounds['maxy'].values)&
                                    (hrrr_00['grid_latt']>perimeter_bounds['miny'].values))
        
        hrrr_00 = hrrr_00.sel(grid_xt=np.unique(grid_xt_sel), grid_yt=np.unique(grid_yt_sel))

        print(np.unique(intersection_sub['col'].values))
        print(np.unique(intersection_sub['row'].values))

        #GET THE POLYGON VARIABLES
        for var_raw in varis_raw:
            df_hrrr_raw[var_raw].iloc[count] = \
                    np.nansum(intersection_sub['weights'].values*hrrr_00.sel(grid_yt = np.unique(intersection_sub['row'].values),
                                                    grid_xt = np.unique(intersection_sub['col'].values))[var_raw].values, axis=(1,2))
         
        #get the winds
        x_interp, y_interp = interpnn(hrrr_00, perp_segs)
        uwind_nearest = hrrr_00['uwind_10m'].values[x_interp,y_interp]
        vwind_nearest = hrrr_00['vwind_10m'].values[x_interp,y_interp]
        speed_nearest = np.sqrt((uwind_nearest**2)+(vwind_nearest**2))
        
        #get the length weights
        fireline_length = segs['geometry'].length
        fireline_length_fraction = fireline_length/(np.nansum(fireline_length))
        
        #get the perpindicular fraction
        #unit_vectors
        uwind_unit = uwind_nearest/np.sqrt((uwind_nearest**2)+(vwind_nearest**2))
        vwind_unit = vwind_nearest/np.sqrt((uwind_nearest**2)+(vwind_nearest**2))
        #what portion of the unit vector is along the perpindicular? this uses cos(x) = u.v/||u||*||v|| vector law
        scale_factor = ((perp_segs['x']*uwind_unit)+
                                 (perp_segs['y']*vwind_unit))/(np.sqrt((perp_segs['x']**2)+(perp_segs['y']**2)))
        scale_factor_zeroed = ((perp_segs['x']*uwind_unit)+
                                 (perp_segs['y']*vwind_unit))/(np.sqrt((perp_segs['x']**2)+(perp_segs['y']**2)))
        scale_factor_zeroed[scale_factor_zeroed<0] =0 
        
    
        hd00 = df_hrrr_raw['vpd_2m'].iloc[count:count+1].values
        #print(hd00)
        #fill in the dataframe
        df_hrrr_derived.loc[count,'day'] = today
        df_hrrr_derived.loc[count, 'hd0w0'] = hd00*df_hrrr_raw['wind_speed'].iloc[count]
        df_hrrr_derived.loc[count, 'hd0w0_edge'] = hd00*np.nanmean(speed_nearest)
        df_hrrr_derived.loc[count, 'hd0w0_edge_lw'] = hd00*(np.nansum(speed_nearest*fireline_length_fraction))
        df_hrrr_derived.loc[count, 'hd0w0_perp'] = hd00*(np.nanmean(speed_nearest*scale_factor))
        df_hrrr_derived.loc[count, 'hd0w0_perp_lw'] = hd00*(np.nansum(speed_nearest*scale_factor*fireline_length_fraction))
        df_hrrr_derived.loc[count, 'hd0w0_perp_out'] = hd00*(np.nanmean(speed_nearest*scale_factor_zeroed))
        df_hrrr_derived.loc[count, 'hd0w0_perp_out_lw'] = hd00*(np.nansum(speed_nearest*scale_factor_zeroed*fireline_length_fraction))

        
        df_hrrr = pd.concat([df_hrrr_derived,df_hrrr_raw],axis=1)
        #print(df_hrrr.iloc[count:count+1])
        #break #to only do the first day
        count = count+1
    return df_hrrr
        

In [ ]:
def hrrr_directional_hourly(df_split, df_poly,df_perim,day_start_hour, time_ave_interval):   
    #do the intersection
    hrrr_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                (df_poly.iloc[ii:ii+1],'HRRR_GRID',0.05) 
                                for ii in range(len(df_poly)))
    fire_hrrr_intersection=gpd.GeoDataFrame(pd.concat(hrrr_intersections, ignore_index=True))
    fire_hrrr_intersection.set_geometry(col='geometry')
    
    #loop over all of the days we have intersections
    times_intersect = np.unique(fire_hrrr_intersection['time'].values)
    times_utc = np.unique(fire_hrrr_intersection['UTC Day'].values)
    
    #preallocate the feature vector
    #set up the output dfs
    varis_raw = ['temp_2m', 'q_2m','wind_speed','veggie', 'dewpt','weasd','soilm', 'esat_2m','e_2m','vpd_2m',
                 'hwp', 'dewpt','veg_term','gust_max_term', 'dd_term','mois_term','snowc_term'] #don't put vpd in here, derived
    df_hrrr_raw = generate_df(varis_raw, 12*len(times_intersect))
    varis_derived = ['day','dhd00w0','dhd05w0','dhd10w0','dhd15w0','dhd20w0','dhd25w0','dhd30w0',
                     'hd00w0','hd05w0','hd10w0','hd15w0','hd20w0','hd25w0','hd30w0',
                     'dhwp']
    df_hrrr_derived = generate_df(varis_derived, 12*len(times_intersect))

    count = 0
    for today in times_intersect:
        print(today)
        
        #get the polygon weights for the timestep
        df_sub = fire_hrrr_intersection.iloc[np.where(fire_hrrr_intersection['time'].values==today)]
        df_sub = df_sub.set_index([str(day_start_hour)+ 'Z Start Day', 'row', 'col'])
        df_sub=df_sub[~df_sub.index.duplicated()]
        intersection_sub = df_sub.to_xarray() #polygon weights for today
        
        #perimeter and bounds for this timestep
        perim_today = df_perim[df_poly['time'].values==today].reset_index()
        split_today = df_split[df_poly['time'].values==today].reset_index()
        perimeter_bounds = perim_today.buffer(0.05).bounds
        
        #get the 12hrs of HRRR for this polygon
        times_back = pd.date_range(start=np.datetime64(today)-np.timedelta64(3,'D'), end=np.datetime64(today)+
                                 np.timedelta64(11,'h'),freq='H') #do now-3days out to now+11hr
        files_back,times_back_used = make_file_namelist(times_back,'/data2/lthapa/ML_daily/pygraf/Processed_HRRR_YYYYMMDDHH.nc')

        dat_hrrr = xr.open_mfdataset(files_back,concat_dim='time',combine='nested',compat='override', coords='all')
        dat_hrrr = dat_hrrr.assign_coords({'time': times_back_used})
        
        # we want all of the hours, so we only select by region
        time_sel,grid_yt_sel, grid_xt_sel = np.where((dat_hrrr['grid_lont']<perimeter_bounds['maxx'].values)&
                                    (dat_hrrr['grid_lont']>perimeter_bounds['minx'].values)&
                                    (dat_hrrr['grid_latt']<perimeter_bounds['maxy'].values)&
                                    (dat_hrrr['grid_latt']>perimeter_bounds['miny'].values))
        hrrr_region = dat_hrrr.sel(time=times_back_used,
                                grid_xt=np.unique(grid_xt_sel),
                                grid_yt=np.unique(grid_yt_sel))
        
        #get perpindicular segments
        segs_list = list(split_today.loc[0,'fireline_split'].geoms)
        segs = gpd.GeoDataFrame({'ind':np.arange(0,len(segs_list)),
                        'time': np.repeat(today,len(segs_list)),
                        'geometry':segs_list})
        segs = segs[segs.length>0.0005]
        perp_segs = find_perp_outside_segments(segs,0.005, perim_today.loc[0,'perimeter']) #get the perpindicular segments
       
        #get the length weights
        fireline_length = segs['geometry'].length
        fireline_length_fraction = fireline_length/(np.nansum(fireline_length))

        #do the wind interpolation
        x_interp, y_interp = interpnn_3d(hrrr_region, perp_segs)
        
        times_back_used = np.array([times_back_used[kk].to_datetime64() for kk in range(len(times_back_used))])
        print(type(np.where(times_back_used>=today)[0]))
        times_today = times_back_used[times_back_used>=today]
        print(times_today)
        
        for hour in times_today:
            print(hour)
            hrrr_region_hour = hrrr_region.sel(time=hour) #select HRRR for the hour
            
            #do the polygon-averaged variables
            for var_raw in varis_raw:
                df_hrrr_raw[var_raw].iloc[count] = \
                    np.nansum(intersection_sub['weights'].values*\
                    hrrr_region_hour.sel(grid_yt = np.unique(intersection_sub['row'].values),
                                        grid_xt = np.unique(intersection_sub['col'].values))[var_raw].values, axis=(1,2))
         
            
            uwind_nearest = hrrr_region['uwind_10m'].sel(time=hour).values[x_interp,y_interp] 
            vwind_nearest = hrrr_region['vwind_10m'].sel(time=hour).values[x_interp,y_interp]

            #get the perpindicular fraction
            #unit_vectors
            uwind_unit = uwind_nearest/np.sqrt((uwind_nearest**2)+(vwind_nearest**2))
            vwind_unit = vwind_nearest/np.sqrt((uwind_nearest**2)+(vwind_nearest**2))
            speed_nearest = np.sqrt((uwind_nearest**2)+(vwind_nearest**2))

            #what portion of the unit vector is along the perpindicular? this uses cos(x) = u.v/||u||*||v|| vector law
            scale_factor = ((perp_segs['x']*uwind_unit)+(perp_segs['y']*vwind_unit))/(np.sqrt((perp_segs['x']**2)+(perp_segs['y']**2)))
            scale_factor_zeroed = ((perp_segs['x']*uwind_unit)+(perp_segs['y']*vwind_unit))/(np.sqrt((perp_segs['x']**2)+(perp_segs['y']**2)))
            scale_factor_zeroed[scale_factor_zeroed<0] =0 
            speed_perp = speed_nearest*scale_factor_zeroed
            
            
            #calculate the components of the derived indices
            dw0 = np.nansum(speed_perp.values*fireline_length_fraction) #directional wind
            w0 = df_hrrr_raw.loc[count,'wind_speed']
            hd00 = np.nansum(intersection_sub['weights'].values*\
                                  hrrr_region.sel(time=hour,
                                                  grid_yt = np.unique(intersection_sub['row'].values),
                                                  grid_xt = np.unique(intersection_sub['col'].values))['vpd_2m'].values, 
                                  axis=(1,2))[0]
            print(hd00,dw0,w0)
            hd05 = np.nansum(intersection_sub['weights'].values*\
                                  hrrr_region.sel(time=hour-np.timedelta64(12,'h'),
                                                  grid_yt = np.unique(intersection_sub['row'].values),
                                                  grid_xt = np.unique(intersection_sub['col'].values))['vpd_2m'].values, 
                                  axis=(1,2))[0]
            hd10 = np.nansum(intersection_sub['weights'].values*\
                                  hrrr_region.sel(time=hour-np.timedelta64(24,'h'),
                                                  grid_yt = np.unique(intersection_sub['row'].values),
                                                  grid_xt = np.unique(intersection_sub['col'].values))['vpd_2m'].values, 
                                  axis=(1,2))[0]
            hd15 = np.nansum(intersection_sub['weights'].values*\
                                  hrrr_region.sel(time=hour-np.timedelta64(36,'h'),
                                                  grid_yt = np.unique(intersection_sub['row'].values),
                                                  grid_xt = np.unique(intersection_sub['col'].values))['vpd_2m'].values, 
                                  axis=(1,2))[0]
            hd20 = np.nansum(intersection_sub['weights'].values*\
                                  hrrr_region.sel(time=hour-np.timedelta64(48,'h'),
                                                  grid_yt = np.unique(intersection_sub['row'].values),
                                                  grid_xt = np.unique(intersection_sub['col'].values))['vpd_2m'].values, 
                                  axis=(1,2))[0]
            hd25 = np.nansum(intersection_sub['weights'].values*\
                                  hrrr_region.sel(time=hour-np.timedelta64(60,'h'),
                                                  grid_yt = np.unique(intersection_sub['row'].values),
                                                  grid_xt = np.unique(intersection_sub['col'].values))['vpd_2m'].values, 
                                  axis=(1,2))[0]
            hd30 = np.nansum(intersection_sub['weights'].values*\
                                  hrrr_region.sel(time=hour-np.timedelta64(72,'h'),
                                                  grid_yt = np.unique(intersection_sub['row'].values),
                                                  grid_xt = np.unique(intersection_sub['col'].values))['vpd_2m'].values, 
                                  axis=(1,2))[0]
            

            
            #calculate the derived variables
            df_hrrr_derived.loc[count,'day'] = hour
            df_hrrr_derived.loc[count,'dhd00w0'] = hd00*dw0
            df_hrrr_derived.loc[count,'dhd05w0'] = hd05*dw0
            df_hrrr_derived.loc[count,'dhd10w0'] = hd10*dw0
            df_hrrr_derived.loc[count,'dhd15w0'] = hd15*dw0
            df_hrrr_derived.loc[count,'dhd20w0'] = hd20*dw0
            df_hrrr_derived.loc[count,'dhd25w0'] = hd25*dw0
            df_hrrr_derived.loc[count,'dhd30w0'] = hd30*dw0
            
            df_hrrr_derived.loc[count,'hd00w0'] = hd00*w0
            df_hrrr_derived.loc[count,'hd05w0'] = hd05*w0
            df_hrrr_derived.loc[count,'hd10w0'] = hd10*w0
            df_hrrr_derived.loc[count,'hd15w0'] = hd15*w0
            df_hrrr_derived.loc[count,'hd20w0'] = hd20*w0
            df_hrrr_derived.loc[count,'hd25w0'] = hd25*w0
            df_hrrr_derived.loc[count,'hd30w0'] = hd30*w0
            
            #process wind just like in the python script
            df_hrrr_derived.loc[count,'dhwp'] = df_hrrr_raw.loc[count,'veg_term']*\
                                                    (np.nansum(speed_perp.values*fireline_length_fraction)**1.11)*\
                                                    (df_hrrr_raw.loc[count,'dd_term']**0.92)*\
                                                    (df_hrrr_raw.loc[count,'mois_term']**6.95)*\
                                                    (df_hrrr_raw.loc[count,'snowc_term'])
                
            
            
            count=count+1
        
    
    df_hrrr_hourly = pd.concat([df_hrrr_derived,df_hrrr_raw],axis=1)
    #df_hrrr_12h = df_hrrr_hourly.set_index('day').resample('12H').mean()
    return(df_hrrr_hourly)

In [ ]:
#nearest neighbor interpolation
#gives x and y values to select
#hrrr needs to be for a given timestep and subsetted to polygon shape
def interpnn(hrrr, perps):
    #code up nearest neighbor
    hrrr_lat_shape = hrrr['grid_latt'].values.shape
    hrrr_lat = np.tile(hrrr['grid_latt'].values,(len(perps),1,1))
    midpoint_lat = np.tile(perps['y_mid'].values,hrrr_lat_shape[0]*hrrr_lat_shape[1]).reshape(hrrr_lat.shape,order='F')
        
    hrrr_lon_shape = hrrr['grid_lont'].values.shape
    hrrr_lon = np.tile(hrrr['grid_lont'].values,(len(perps),1,1))
    midpoint_lon = np.tile(perps['x_mid'].values,hrrr_lon_shape[0]*hrrr_lon_shape[1]).reshape(hrrr_lon.shape,order='F')
        
    distances = np.sqrt(((hrrr_lon-midpoint_lon)**2)+((hrrr_lat-midpoint_lat)**2))
    indices = np.array([np.where(distances==np.nanmin(distances,axis=(1,2))[kk])for kk in range(len(perps))])
    x_nearest_sel = np.squeeze(indices[:,1,:])
    y_nearest_sel = np.squeeze(indices[:,2,:])
    return x_nearest_sel, y_nearest_sel

In [ ]:
 fig,ax=plt.subplots(figsize=(15,15))
        perimeter.plot(ax=ax,color='r',alpha=0.5)
        segs.plot(ax=ax,column='ind',linewidth=2,legend=True,cmap='tab20_r')
        perp_segs.plot(ax=ax,color='k')
        plt.quiver(hrrr_subdaily_mean_wind_region['grid_lont'], hrrr_subdaily_mean_wind_region['grid_latt'], hrrr_subdaily_mean_wind_region['uwind_10m'], hrrr_subdaily_mean_wind_region['vwind_10m'])
        plt.title(str(today))
        plt.show()
        
        print(interp_at_perps(hrrr_subdaily_mean_wind_region,perp_segs,'uwind_10m'))
        print(interp_at_perps(hrrr_subdaily_mean_wind_region,perp_segs,'vwind_10m'))
        print(interp_at_perps(hrrr_subdaily_mean_wind_region,perp_segs,'wind_speed'))

        print('hi')
        

In [ ]:
#input HRRR at a given timestep (today)
def calculate_perp_ws(hrrr,perps):
     #interpolate the u and v components, for direction with scipy
    unew = interp_at_perps(hrrr,perps,'uwind_10m')
    vnew = interp_at_perps(hrrr,perps,'vwind_10m')
    speednew = interp_at_perps(hrrr,perps,'wind_speed')
    
    #unit_vectors
    unew_unit = unew/np.sqrt((unew**2)+(vnew**2))
    vnew_unit = vnew/np.sqrt((unew**2)+(vnew**2))
    #what portion of the unit vector is along the perpindicular? this uses cos(x) = u.v/||u||*||v|| vector law
    scale_factor = ((perps['x']*unew_unit)+
                                 (perps['y']*vnew_unit))/(np.sqrt((perps['x']**2)+(perps['y']**2)))
    scale_factor[scale_factor<0] =0    
    perp_wind_speed = scale_factor*speednew
    return perp_wind_speed

In [ ]:
#input HRRR at a given timestep (today),  the perpindiculars, and the name of the variable you want to interpolate
def interp_at_perps(hrrr,perps,var_interp):
     #interpolate the u and v components, for direction with scipy
    f_var = si.interp2d(hrrr['grid_lont'].values.flatten(), 
                      hrrr['grid_latt'].values.flatten(), 
                      hrrr[var_interp].values.flatten(),kind='linear')
    var_new = np.array([f_var(perps['x_mid'].values[kk], perps['y_mid'].values[kk])[0] 
                     for kk in range(len(perps))])
    return var_new

In [ ]:
#IN df_segs, a geodataframe of the line segments making up the fire front
#IN d, the length of the resulting perpindicular segments
#OUT 
def find_perp_outside_segments(df_segs,d,perim):
    #allocate the 
    varis_seg = ['time','x_mid','y_mid','x_outside','y_outside']
    perpindicular_segments = generate_df(varis_seg, len(df_segs))
    perpindicular_segments['time'] = df_segs['time'].values
    
    #get the start and end points of each line
    x1 =np.array([list(df_segs['geometry'].iloc[ll].coords)[0][0] for ll in range(len(df_segs))])
    y1 =np.array([list(df_segs['geometry'].iloc[ll].coords)[0][1] for ll in range(len(df_segs))])
    x2 =np.array([list(df_segs['geometry'].iloc[ll].coords)[1][0] for ll in range(len(df_segs))])
    y2 =np.array([list(df_segs['geometry'].iloc[ll].coords)[1][1] for ll in range(len(df_segs))])
    
    #get the midpoints
    x_mid = (x1+x2)/2
    y_mid = (y1+y2)/2
    perpindicular_segments['x_mid'] = x_mid
    perpindicular_segments['y_mid'] = y_mid
    
    
    #get the slopes and perp slope of each segment
    m_line = (y2-y1)/(x2-x1)
    m_perp = -1/m_line
    
    #get the inward and outward pointing perpindiculars
    x_mid_plus = x_mid+(d/np.sqrt(1+(m_perp**2)))
    y_mid_plus = (m_perp*(x_mid_plus-x_mid)) + y_mid
    x_mid_minus = x_mid-(d/np.sqrt(1+(m_perp**2)))
    y_mid_minus = (m_perp*(x_mid_minus-x_mid)) + y_mid
    
    #when m_line = 0 (just add d to the y direction)
    x_mid_plus[m_line==0] = x_mid[m_line==0]
    x_mid_minus[m_line==0] = x_mid[m_line==0]
    y_mid_plus[m_line==0] = y_mid[m_line==0]+d
    y_mid_minus[m_line==0] = y_mid[m_line==0]-d
    
    #turn them into points
    plus_points = [Point(list(zip(x_mid_plus,y_mid_plus))[mm]) for mm in range(len(df_segs))]
    minus_points = [Point(list(zip(x_mid_minus,y_mid_minus))[mm]) for mm in range(len(df_segs))]
    
    x_outside = np.zeros(len(df_segs))
    y_outside = np.zeros(len(df_segs))
    
    #check which points are in the polygons, drop those
    for xx in range(len(df_segs)):
        #print(perim['geometry'].iloc[0])
        if not perim.contains(plus_points[xx]):
            #print(xx)
            x_outside[xx] = x_mid_plus[xx]
            y_outside[xx] =y_mid_plus[xx]
        if not perim.contains(minus_points[xx]):
            x_outside[xx] = x_mid_minus[xx]
            y_outside[xx] = y_mid_minus[xx]
            
    perpindicular_segments['x_outside']=x_outside
    perpindicular_segments['y_outside']=y_outside

    perpindicular_segments['x'] = x_outside-x_mid
    perpindicular_segments['y'] = y_outside-y_mid
    #add the outwrd pointing perp as the main tracked geometry
    perpindicular_segments['perpindicular_lines'] = [LineString(((perpindicular_segments['x_mid'].iloc[xx], perpindicular_segments['y_mid'].iloc[xx]),
                                         (perpindicular_segments['x_outside'].iloc[xx], perpindicular_segments['y_outside'].iloc[xx]))) 
                                             for xx in range(len(perpindicular_segments))] 
    perpindicular_segments = perpindicular_segments.set_geometry('perpindicular_lines')
    return perpindicular_segments

In [ ]:
def make_hrrr_file_namelist(time):
    base_filename = '/data2/lthapa/ML_daily/pygraf/Processed_HRRR_YYYYMMDDHH.nc'
    base_filename_list = np.array([])
    times_back_used = np.array([])

    
    for jj in range(len(time)):
        fname = base_filename.replace('YYYYMMDDHH',time[jj].strftime('%Y%m%d%H'))
        if exists(fname):
            base_filename_list = np.append(base_filename_list,fname)
            times_back_used = np.append(times_back_used,time[jj])
            
    return base_filename_list, times_back_used

### RAVE

In [1]:
def rave_timeseries(df, day_start_hour, sum_interval):
    varis = ['Mean_FRP', 'FRE', 'CO2', 'CO', 'SO2', 'OC', 'BC', 'PM2.5', 'NOx', 'NH3'] #don't need 'area', it's the area of each cell

    #do the intersection, in parallel
    rave_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (df.iloc[ii:ii+1],'RAVE_GRID',0.1) 
                                 for ii in range(len(df)))
    fire_rave_intersection=gpd.GeoDataFrame(pd.concat(rave_intersections, ignore_index=True))
    fire_rave_intersection.set_geometry(col='geometry')    
    fire_rave_intersection = fire_rave_intersection.set_index(['time', 'row', 'col'])
    fire_rave_intersection=fire_rave_intersection[~fire_rave_intersection.index.duplicated()]

    fire_rave_intersection_xr = fire_rave_intersection.to_xarray()
    
    #load in rave data associated with the fire
    times = pd.date_range(np.datetime64(df[str(day_start_hour)+ 'Z Start Day'].iloc[0]),
                        np.datetime64(df[str(day_start_hour)+ 'Z Start Day'].iloc[len(df)-1])+
                        np.timedelta64(1,'D'))
    rave_filenames,times_back_used = make_file_namelist(times,'/data2/lthapa/YYYY/AprYYYY_to_OctYYYY/Hourly_Emissions_FV3_13km_YYYYMMDD0000_YYYYMMDD2300xr.nc')
    dat_rave = xr.open_mfdataset(rave_filenames,concat_dim='Time',combine='nested',compat='override', coords='all')
    dat_rave = dat_rave.assign_coords({'Time': dat_rave.time}) #assign coords so we can resample along time
    dat_rave = dat_rave.resample(Time=str(sum_interval)+'H',base=day_start_hour).sum(dim='Time') #take the daily sum
    
    print(fire_rave_intersection_xr['time'].values)
    print(dat_rave['Time'].values)
    #select the locations and times we want
    dat_rave_sub = dat_rave.isel(yFRP = fire_rave_intersection_xr['row'].values.astype(int), 
                    xFRP = fire_rave_intersection_xr['col'].values.astype(int)).sel(
                    Time = pd.to_datetime(fire_rave_intersection_xr['time'].values))#these should be lined up correctly

    ndays = len(fire_rave_intersection_xr['time'])
    
    #preallocate space for the output
    df_rave = pd.DataFrame({'day':np.zeros(ndays),'Mean_FRP':np.zeros(ndays),\
                          'FRE':np.zeros(ndays),  'CO2':np.zeros(ndays),  'CO':np.zeros(ndays),\
                          'SO2':np.zeros(ndays),  'OC':np.zeros(ndays),  'BC':np.zeros(ndays),\
                          'PM2.5':np.zeros(ndays),  'NOx':np.zeros(ndays),  'NH3':np.zeros(ndays)})

    df_rave['day'].iloc[:] = pd.to_datetime(fire_rave_intersection_xr['time'].values)
    for var in varis:
        df_rave[var] = np.nansum(fire_rave_intersection_xr['weights'].values*dat_rave_sub[var].values, axis=(1,2))
    
    return df_rave

In [2]:
def make_rave_file_namelist(time):
    base_filename = '/data2/lthapa/YEAR/AprYEAR_to_OctYEAR/Hourly_Emissions_FV3_13km_FULLDATE0000_FULLDATE2300xr.nc'
    base_filename_list = np.repeat(base_filename, len(time))

    
    for jj in range(len(time)):
        base_filename_list[jj] = base_filename_list[jj].replace('YEAR',time[jj].strftime('%Y')).\
                                    replace('FULLDATE',time[jj].strftime('%Y%m%d'))
    return base_filename_list

### PWS

In [2]:
def pws_timeseries(df, day_start_hour):
    #do the intersection, in parallel
    tic = datetime.datetime.now()
    pws_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (fire_daily.iloc[ii:ii+1],'PWS_GRID',0.05) 
                                 for ii in range(len(fire_daily)))
    toc = datetime.datetime.now()
    print(toc-tic)
    fire_pws_intersection=gpd.GeoDataFrame(pd.concat(pws_intersections, ignore_index=True))
    fire_pws_intersection.set_geometry(col='geometry')
    fire_pws_intersection = fire_pws_intersection.set_index([str(day_start_hour)+ 'Z Start Day', 'lat', 'lon'])
    
    fire_pws_intersection_xr = fire_pws_intersection.to_xarray()
    
    #load in PWS data associated with the fire (it's only one dataset)
    
    #open the PWS files
    path_pws = '/data2/lthapa/PWS_6_jan_2021.nc'
    dat_pws = xr.open_dataset(path_pws) #map is fixed in time
    #print(dat_pws)
    
    dat_pws = dat_pws.assign_coords({'time': pd.to_datetime(fire_pws_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values)})
    dat_pws_daily = dat_pws['Band1'].expand_dims({'time': pd.to_datetime(fire_pws_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values)}) #the PWS expanded over all the days
    
    dat_pws_daily_sub = dat_pws_daily.sel(lat = fire_pws_intersection_xr['lat'].values, 
                                          lon = fire_pws_intersection_xr['lon'].values,
                      time = pd.to_datetime(fire_pws_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values), method='nearest')
                                          
    ndays = len(fire_pws_intersection_xr[str(day_start_hour)+ 'Z Start Day'])
    
    #preallocate space for the output
    df_pws = pd.DataFrame({'day':np.zeros(ndays),'PWS':np.zeros(ndays)})

    df_pws['day'].iloc[:] = pd.to_datetime(fire_pws_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values)
    varis=['PWS']
    for var in varis:
        df_pws[var] = np.nansum(fire_pws_intersection_xr['weights'].values*dat_pws_daily_sub.values, axis=(1,2))
    
    return df_pws
    

### HRRR IGBP Vegetation

In [ ]:
def hrrr_igbp_timeseries(df,day_start_hour):  
    df_igbp = pd.DataFrame({'day': np.zeros(len(df)),'Fraction_Evergreen_Needleleaf_Forests':np.zeros(len(df)),
                            'Fraction_Evergreen_Broadleaf_Forests':np.zeros(len(df)), 
                            'Fraction_Deciduous_Needleleaf_Forests':np.zeros(len(df)),
                            'Fraction_Deciduous_Broadleaf_Forests':np.zeros(len(df)), 
                            'Fraction_Mixed_Forests':np.zeros(len(df)),
                            'Fraction_Closed_Shrublands':np.zeros(len(df)),
                            'Fraction_Open_Shrublands':np.zeros(len(df)),
                            'Fraction_Woody_Savannas':np.zeros(len(df)),
                            'Fraction_Savannas':np.zeros(len(df)),
                            'Fraction_Grasslands':np.zeros(len(df)),
                            'Fraction_Permanent_Wetlands':np.zeros(len(df)),
                            'Fraction_Croplands':np.zeros(len(df)),
                            'Fraction_Urban_and_Builtup_Lands':np.zeros(len(df)),
                            'Fraction_Cropland_Natural_Vegetation_Mosaics':np.zeros(len(df)),
                            'Fraction_Permanent_Snow_and_Ice':np.zeros(len(df)),
                            'Fraction_Barren':np.zeros(len(df)),
                            'Fraction_Water_Bodies':np.zeros(len(df))})
    
    varis = ['Evergreen Needleleaf Forests','Evergreen Broadleaf Forests', 'Deciduous Needleleaf Forests',
         'Deciduous Broadleaf Forests','Mixed Forests','Closed Shrublands', 'Open Shrublands', 'Woody Savannas',
         'Savannas','Grasslands', 'Permanent Wetlands','Croplands','Urban and Builtup Lands', 
         'Cropland Natural Vegetation Mosaics','Permanent Snow and Ice','Barren','Water Bodies']

        
    #do the intersection, in parallel
    tic=time.time()
    hrrr_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (fire_daily.iloc[ii:ii+1],'HRRR_GRID',0.05) 
                                 for ii in range(len(fire_daily)))
    fire_hrrr_intersection=gpd.GeoDataFrame(pd.concat(hrrr_intersections, ignore_index=True))
    fire_hrrr_intersection.set_geometry(col='geometry')
    toc = time.time()
    print(toc-tic)
    fire_hrrr_intersection = fire_hrrr_intersection.set_index([str(day_start_hour)+ 'Z Start Day', 'row', 'col'])
    fire_hrrr_intersection_xr = fire_hrrr_intersection.to_xarray()
    print(fire_hrrr_intersection_xr['weights'].sum(dim=str(day_start_hour)+ 'Z Start Day'))
    
    #load in all the HRRR days we want
    times = pd.date_range(np.datetime64(df[str(day_start_hour)+ 'Z Start Day'].iloc[0]),
                        np.datetime64(df[str(day_start_hour)+ 'Z Start Day'].iloc[len(df)-1])+
                        np.timedelta64(1,'D'))
    files,times_used = make_file_namelist(times,'/data2/lthapa/ML_daily/pygraf/Processed_HRRR_YYYYMMDDHH.nc')
    dat_hrrr = xr.open_mfdataset(files,concat_dim='time',combine='nested',compat='override', coords='all')
    dat_hrrr = dat_hrrr.assign_coords({'time': times})

    #select by time and location
    hrrr_daily_mean = dat_hrrr['veggie'].resample(time='24H',base=day_start_hour, label='left').mean(dim='time') #take the daily mean        
        
    hrrr_daily_mean_region = hrrr_daily_mean.sel(grid_yt = np.unique(fire_hrrr_intersection_xr['row'].values),
                                                 grid_xt = np.unique(fire_hrrr_intersection_xr['col'].values),
                                                 time = pd.to_datetime(fire_hrrr_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values+
                                                 'T'+str(day_start_hour)+':00:00')) #get the location of the overlaps
    
    
    df_igbp['day'] = pd.to_datetime(fire_hrrr_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values+
                                                 'T'+str(day_start_hour)+':00:00')
    
    weights_fire = fire_hrrr_intersection_xr['weights'].values
    veggie_fire = hrrr_daily_mean_region.values
    
    for vt in range(1,18):
        weights_by_veg_type= np.where(veggie_fire==vt,weights_fire,0)
        #print(np.nansum(weights_by_veg_type,axis=(1,2))) #this is what we want to return
        df_igbp['Fraction_'+varis[vt-1].replace(' ', '_')] = np.nansum(weights_by_veg_type,axis=(1,2))
        
    
    
    df_igbp_choose_max = df_igbp[['Fraction_Evergreen_Needleleaf_Forests','Fraction_Evergreen_Broadleaf_Forests', 'Fraction_Deciduous_Needleleaf_Forests',
                            'Fraction_Deciduous_Broadleaf_Forests', 'Fraction_Mixed_Forests','Fraction_Closed_Shrublands',
                            'Fraction_Open_Shrublands','Fraction_Woody_Savannas','Fraction_Savannas',
                            'Fraction_Grasslands','Fraction_Permanent_Wetlands','Fraction_Croplands',
                            'Fraction_Urban_and_Builtup_Lands','Fraction_Cropland_Natural_Vegetation_Mosaics','Fraction_Permanent_Snow_and_Ice',
                            'Fraction_Barren','Fraction_Water_Bodies']]
    
    return df_igbp

   

### ESI

In [18]:
def esi_timeseries(df, day_start_hour):
    #preallocate space for the output
    df_esi = pd.DataFrame({'day':np.zeros(len(df)),'ESI':np.zeros(len(df))})
    
    #do the intersection, in parallel
    #tic=time.time()
    esi_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (fire_daily.iloc[ii:ii+1],'ESI_GRID',0.05) 
                                 for ii in range(len(fire_daily)))
    #toc = time.time()
    #print(toc-tic)
    
    fire_esi_intersection=gpd.GeoDataFrame(pd.concat(esi_intersections, ignore_index=True))
    fire_esi_intersection.set_geometry(col='geometry')
    
    
    fire_esi_intersection = fire_esi_intersection.set_index([str(day_start_hour)+ 'Z Start Day', 'lat', 'lon'])
    
    fire_esi_intersection_xr = fire_esi_intersection.to_xarray()
    print(fire_esi_intersection_xr['weights'].sum(dim=['lat','lon']))
    

    #load in esi data associated with the fire
    times = pd.date_range(np.datetime64(df[str(day_start_hour)+ 'Z Start Day'].iloc[0]),
                        np.datetime64(df[str(day_start_hour)+ 'Z Start Day'].iloc[len(df)-1])+
                        np.timedelta64(1,'D'))
    esi_filenames, esi_times = make_file_namelist(times,'/data2/lthapa/2020/ESI/DFPPM_4WK_YYYYJJJ.nc')
    
    print(esi_filenames)
    print(esi_times)
    
    
    #open the esi files
    dat_esi = xr.open_mfdataset(esi_filenames,concat_dim='time',combine='nested',compat='override', coords='all')
    dat_esi = dat_esi.assign_coords({'time': esi_times}) #assign coords so we can resample along time
    dat_esi = dat_esi.where(dat_esi['Band1']!=-9999)
    dat_esi_daily = dat_esi.reindex(time=times,method='nearest')
    dat_esi_daily_sub = dat_esi_daily.sel(lat = fire_esi_intersection_xr['lat'].values, 
                                          lon = fire_esi_intersection_xr['lon'].values,
                      time = pd.to_datetime(fire_esi_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values), method='nearest')
                                          

    
    

    df_esi['day'].iloc[:] = pd.to_datetime(fire_esi_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values)
    varis=['ESI']
    for var in varis:
        df_esi[var] = np.nansum(fire_esi_intersection_xr['weights'].values*dat_esi_daily_sub['Band1'].values, axis=(1,2))
    
    return df_esi
    

### Resources/Personel (different format because database is excel spreadsheets, not netcdf files)

In [ ]:
def resources_timeseries(df, day_start_hour,sit209_data):

    #get the fire incident number, lat, and lon
    incident_number = df['Incident Number'].iloc[0]
    fire_lat = df['Lat Fire'].iloc[0]
    fire_lon = df['Lon Fire'].iloc[0]
    #print(incident_number, fire_lat, fire_lon)
    
    sit209_data_fire = sit209_data[sit209_data['INC_IDENTIFIER']==incident_number]
    #print(sit209_data_fire)
    #do the time zone conversion
    obj=TimezoneFinder() #initialize the timezone finder
    tz = obj.timezone_at(lng=fire_lon, lat=fire_lat) #get the timezone
    local = pytz.timezone(tz)
    utc = pytz.utc
    
    #put the start and end times in local time
    loc_dt_start = [local.localize(datetime.strptime(date, '%m/%d/%Y %H:%M:%S %p')) for date in sit209_data_fire['REPORT_FROM_DATE'].values]
    loc_dt_end = [local.localize(datetime.strptime(date, '%m/%d/%Y %H:%M:%S %p')) for date in sit209_data_fire['REPORT_TO_DATE'].values]
    
    #put them in UTC time
    utc_dt_start = [time_start.astimezone(utc) for time_start in loc_dt_start]
    utc_dt_end = [time_end.astimezone(utc) for time_end in loc_dt_end]
    
    start_day = pd.to_datetime(utc_dt_start[0]).strftime('%Y-%m-%d')+' '+str(day_start_hour)+':00'
    
    
    #reassign to UTC time, this DOES keep track of daylight savings (eg +7 is used for PDT, +8 is used for PST)
    sit209_data_fire['Report Start UTC'] = pd.to_datetime(utc_dt_start)
    sit209_data_fire['Report End UTC'] = pd.to_datetime(utc_dt_end)
    sit209_data_fire['Timezone']= tz
    
    #localise the index
    sit209_data_fire = sit209_data_fire.set_index(['Report Start UTC']).tz_localize(None)
    #print(sit209_data_fire.iloc[0:4])
    
    
    ## do the 12z-12z day grouping, based on the UTC times
    #start_day_utc = str(utc_dt_start[0])
    start_day_utc=str(df[str(day_start_hour)+'Z Start Day'][0])
    start_datetime_utc = np.datetime64(start_day_utc[0:10]+'T'+str(day_start_hour).zfill(2)+':00')
    print(start_datetime_utc)
    #sit209_data_fire = sit209_data_fire.resample('24H',origin=start_datetime_utc)

    personnel = sit209_data_fire['RESOURCE_PERSONNEL'].resample('24H',origin=start_datetime_utc).sum().reset_index()
    percent_contained = sit209_data_fire['PCT_CONTAINED_COMPLETED'].resample('24H',origin=start_datetime_utc).mean().reset_index()
    df_sit209 = pd.concat([percent_contained,personnel.drop(columns='Report Start UTC')],axis=1)
    df_sit209.columns=['day', 'percent_contained', 'personnel']
    df_sit209['day'] = pd.to_datetime(df_sit209['day'].values).strftime('%Y-%m-%d')
    inds = df_sit209['day'].isin(df[str(day_start_hour)+'Z Start Day']).values
    
    return df_sit209[inds]
    

### Fuel Loading (1km

In [ ]:
def fuel_loading_timeseries(df, day_start_hour):
    #do the intersection, in parallel
    tic = time.time()
    fuel_loading_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (fire_daily.iloc[ii:ii+1],'FUEL_LOADING_GRID',0.05) 
                                 for ii in range(len(fire_daily)))
    toc = time.time()
    print(toc-tic)
    
    fire_fuel_loading_intersection=gpd.GeoDataFrame(pd.concat(fuel_loading_intersections, ignore_index=True))
    fire_fuel_loading_intersection.set_geometry(col='geometry')
    fire_fuel_loading_intersection = fire_fuel_loading_intersection.set_index([str(day_start_hour)+ 'Z Start Day', 'lat', 'lon'])
    
    fire_fuel_loading_intersection_xr = fire_fuel_loading_intersection.to_xarray()
    
    #load in fuel load data associated with the fire (it's only one dataset)
    
    #open the fuel loading file
    path_fuel_loading = '/data/lthapa/GSL_Internship/fuel_loading_FCCS16.nc'
    dat_fuel_loading = xr.open_dataset(path_fuel_loading) #map is fixed in time
    
    
    dat_fuel_loading = dat_fuel_loading.assign_coords({'time': pd.to_datetime(fire_fuel_loading_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values)})
    dat_fuel_loading = dat_fuel_loading.where(dat_fuel_loading['fuel_loadings']!=-9999)    
    dat_fuel_loading_daily = dat_fuel_loading['fuel_loadings'].expand_dims({'time': pd.to_datetime(fire_fuel_loading_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values)}) #the PWS expanded over all the days
    
    dat_fuel_loading_daily_sub = dat_fuel_loading_daily.sel(lat = fire_fuel_loading_intersection_xr['lat'].values, 
                                          lon = fire_fuel_loading_intersection_xr['lon'].values,
                      time = pd.to_datetime(fire_fuel_loading_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values), method='nearest')
                                          
    ndays = len(fire_fuel_loading_intersection_xr[str(day_start_hour)+ 'Z Start Day'])
    
    #preallocate space for the output
    df_fuel_loading = pd.DataFrame({'day':np.zeros(ndays),'fuel_loadings':np.zeros(ndays)})

    df_fuel_loading['day'].iloc[:] = pd.to_datetime(fire_fuel_loading_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values)
    varis=['fuel_loadings']
    for var in varis:
        df_fuel_loading[var] = np.nansum(fire_fuel_loading_intersection_xr['weights'].values*dat_fuel_loading_daily_sub.values, axis=(1,2))
    
    return df_fuel_loading
    

### Fire Area (ha and ac)

In [ ]:
def fire_area_timeseries(df,day_start_hour):
    df_fire_area = df[[str(start_time)+'Z Start Day','fire area (ha)']].rename(columns={str(start_time)+'Z Start Day':'day'})
    df_fire_area['fire area (m2)'] = df_fire_area['fire area (ha)']*10000
    df_fire_area['fire area (acre)'] = df_fire_area['fire area (ha)']*2.471
    return df_fire_area

### Fuel Loading

In [ ]:
def fuel_loading_timeseries(df,day_start_hour):
    

### FWI IMERG

In [2]:
def imerg_fwi_timeseries(df, day_start_hour):
    varis = ['day','IMERG.FINAL.v6_DC','IMERG.FINAL.v6_DMC','IMERG.FINAL.v6_FFMC',
             'IMERG.FINAL.v6_ISI','IMERG.FINAL.v6_BUI','IMERG.FINAL.v6_FWI',
             'IMERG.FINAL.v6_DSR'] 
    df_imerg = generate_df(varis, len(df))

    #do the intersection, in parallel
    fwi_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (df.iloc[ii:ii+1],'IMERG_FWI_GRID',0.1) 
                                 for ii in range(len(df)))
    fire_fwi_intersection=gpd.GeoDataFrame(pd.concat(fwi_intersections, ignore_index=True))
    fire_fwi_intersection.set_geometry(col='geometry')    
    fire_fwi_intersection = fire_fwi_intersection.set_index([str(day_start_hour)+ 'Z Start Day', 'lat', 'lon'])

    fire_fwi_intersection=fire_fwi_intersection[~fire_fwi_intersection.index.duplicated()]
    
    fire_fwi_intersection_xr = fire_fwi_intersection.to_xarray()

    
    #load in FWI data associated with the fire
    times = pd.date_range(np.datetime64(df['UTC Day'].iloc[0]),
                        np.datetime64(df['UTC Day'].iloc[len(df)-1])+
                        np.timedelta64(1,'D'))
    fwi_filenames,times_back_used = make_file_namelist(times,'/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.YYYYMMDD.nc')
    
    dat_fwi = xr.open_mfdataset(fwi_filenames,concat_dim='time',combine='nested',compat='override', coords='all')
    dat_fwi = dat_fwi.assign_coords({'time': times_back_used}) #assign coords so we can resample along time
    dat_fwi = dat_fwi.resample(time='1H').pad() #make the data hourly, so we can define the day as 12z-12z instead of 0z-0z
    dat_fwi_mean = dat_fwi.resample(time='24H',base=day_start_hour ,label='left').mean(dim='time') #take the daily mean         
    
    #select the locations and times we want
    dat_fwi_sub = dat_fwi_mean.isel(lat = fire_fwi_intersection_xr['lat'].values.astype(int), 
                    lon = fire_fwi_intersection_xr['lon'].values.astype(int)).sel(
                    time = pd.to_datetime(fire_fwi_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values+ ' '+
                                         str(day_start_hour)+':00:00'), method='nearest')#these should be lined up correctly


    df_imerg['day'].iloc[:] = pd.to_datetime(fire_fwi_intersection_xr['12Z Start Day'].values)
    for var in varis[1:len(varis)]:
        df_imerg[var] = np.nansum(fire_fwi_intersection_xr['weights'].values*dat_fwi_sub[var].values, axis=(1,2))
    
    return df_imerg

### FWI IMERG, not averaged

In [1]:
def imerg_fwi_timeseries(df, day_start_hour):
    varis = ['day','IMERG.FINAL.v6_DC','IMERG.FINAL.v6_DMC','IMERG.FINAL.v6_FFMC',
             'IMERG.FINAL.v6_ISI','IMERG.FINAL.v6_BUI','IMERG.FINAL.v6_FWI',
             'IMERG.FINAL.v6_DSR'] 
    df_imerg = generate_df(varis, len(df))

    #do the intersection, in parallel
    fwi_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (df.iloc[ii:ii+1],'IMERG_FWI_GRID',0.1) 
                                 for ii in range(len(df)))
    fire_fwi_intersection=gpd.GeoDataFrame(pd.concat(fwi_intersections, ignore_index=True))
    fire_fwi_intersection.set_geometry(col='geometry')    
    fire_fwi_intersection = fire_fwi_intersection.set_index([str(day_start_hour)+ 'Z Start Day', 'lat', 'lon'])

    fire_fwi_intersection=fire_fwi_intersection[~fire_fwi_intersection.index.duplicated()]
    
    fire_fwi_intersection_xr = fire_fwi_intersection.to_xarray()

    
    #load in FWI data associated with the fire
    times = pd.date_range(np.datetime64(df['UTC Day'].iloc[0]),
                        np.datetime64(df['UTC Day'].iloc[len(df)-1])+
                        np.timedelta64(1,'D'))
    fwi_filenames,times_back_used = make_file_namelist(times,'/data2/lthapa/2020/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.YYYYMMDD.nc')
    
    dat_fwi = xr.open_mfdataset(fwi_filenames,concat_dim='time',combine='nested',compat='override', coords='all')
    dat_fwi = dat_fwi.assign_coords({'time': times_back_used}) #assign coords so we can resample along time
    
    #select the locations and times we want
    dat_fwi_sub = dat_fwi.isel(lat = fire_fwi_intersection_xr['lat'].values.astype(int), 
                    lon = fire_fwi_intersection_xr['lon'].values.astype(int)).sel(
                    time = pd.to_datetime(fire_fwi_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values+ ' '+
                                         str(day_start_hour)+':00:00'), method='nearest')#these should be lined up correctly


    df_imerg['day'].iloc[:] = pd.to_datetime(fire_fwi_intersection_xr['12Z Start Day'].values)
    for var in varis[1:len(varis)]:
        df_imerg[var] = np.nansum(fire_fwi_intersection_xr['weights'].values*dat_fwi_sub[var].values, axis=(1,2))
    
    return df_imerg

## Dataset-Independent Helper Functions

In [3]:
#makes and saves a geodataframe of a grid given the center and corner points for that grid as 2D matrices
def build_one_gridcell(LAT_COR, LON_COR, LAT_CTR, LON_CTR, loc):
    ii=loc[0]
    jj=loc[1]

    #print(LAT_CTR[ii,jj], LON_CTR[ii,jj]) #ctr
    sw = (LON_COR[ii, jj],LAT_COR[ii, jj]) #SW
    se =(LON_COR[ii, jj+1],LAT_COR[ii, jj+1]) #SE
    nw = (LON_COR[ii+1, jj],LAT_COR[ii+1, jj]) #NW
    ne = (LON_COR[ii+1, jj+1],LAT_COR[ii+1, jj+1]) #NE
            
    poly_cell = Polygon([sw,nw,ne,se])

    return LAT_CTR[ii,jj], LON_CTR[ii,jj],ii,jj,poly_cell
    

In [4]:
#poly is the polygon for one timestep (in lcc)
#grid is an xarray of a model grid from the nc file
#grid_names is a string array [0:'lat_center_name',1:'lon_center_name',2:'lat_corner_name',3:'lon_corner_name']

def calculate_intersection(poly,dataset_name,bf):
    print(poly['12Z Start Day'])
    #load in the merra grid
    grid = xr.open_dataset(dataset_name+'.nc')

    #get the bounds of the buffered polygons
    poly_latlon =poly.to_crs(epsg=4326)
    bounds = poly_latlon.buffer(bf).bounds
    
    #first check for rows and cols, filtering near the polygon
    [rows,cols] = np.where((grid.LAT_CTR.values>bounds['miny'].values)&
                    (grid.LAT_CTR.values<bounds['maxy'].values)&
                    (grid.LON_CTR.values>bounds['minx'].values)&
                    (grid.LON_CTR.values<bounds['maxx'].values))
    #print(rows,cols)
    
    locs = zip(rows,cols)
    #print([loc for loc in locs])
    
    """
    if (rows.size==0)|(cols.size==0):
        lat_middle = (bounds['maxy'].values+bounds['miny'].values)/2
        lon_middle = (bounds['maxx'].values+bounds['minx'].values)/2
        
        distance = np.sqrt((grid.LAT_CTR.values-lat_middle)**2+(grid.LON_CTR.values-lon_middle)**2)
        row_minloc,col_minloc = np.where(distance==np.min(distance))
        print(row_minloc,col_minloc)
        
        rows = np.arange(row_minloc-1,row_minloc+2,1)
        cols = np.arange(col_minloc-1,col_minloc+2,1)
        
        locs = [(row,col) for row in rows for col in cols]
        
    #print(rows,cols)
    #print(locs)
    """
    
    #make a geodataframe (in paralell of the rows and cols)
    results = Parallel(n_jobs=8)(delayed(build_one_gridcell)
                                 (grid['LAT_COR'].values, grid['LON_COR'].values,
                                  grid['LAT_CTR'].values, grid['LON_CTR'].values,loc) 
                                 for loc in locs)
    
    #format the grid subset into a dataframs
    df_grid=gpd.GeoDataFrame(results)
    df_grid.columns = ['lat', 'lon', 'row', 'col', 'geometry']
    df_grid.set_geometry(col='geometry',inplace=True,crs='EPSG:4326') #need to say it's in lat/lon before transform to LCC
    df_grid=df_grid.to_crs(epsg=3347)

    
    #intersect the polygon with the grid subset
    intersection = gpd.overlay(df_grid, poly, how='intersection',keep_geom_type=False).drop_duplicates()
    intersection['grid intersection area (ha)'] =intersection['geometry'].area/10000
    intersection['weights'] = intersection['grid intersection area (ha)']/intersection['fire area (ha)'] 
    
    return intersection


In [5]:
#LAT and LON are 2d arrays
def calculate_grid_cell_corners(LAT, LON):
    #we will assume the very edges of the polygons don't touch the boundary of the domain
    lat_corners = (LAT[0:(LAT.shape[0]-1),  0:(LAT.shape[1])-1] + LAT[1:(LAT.shape[0]), 1:(LAT.shape[1])])/2
    lon_corners = (LON[0:(LAT.shape[0]-1),  0:(LAT.shape[1])-1] + LON[1:(LAT.shape[0]), 1:(LAT.shape[1])])/2
    return lat_corners, lon_corners


In [6]:
def make_file_namelist(time,base_filename):
    filename_list = np.array([])
    times_back_used = np.array([])
    for jj in range(len(time)):
        fname = base_filename.replace('YYYY',time[jj].strftime('%Y')).\
                                replace('MM',time[jj].strftime('%m')).\
                                replace('DD',time[jj].strftime('%d')).\
                                replace('HH',time[jj].strftime('%H')).\
                                replace('JJJ',time[jj].strftime('%j'))
        #print(fname)
        if exists(fname):
            filename_list = np.append(filename_list,fname)
            times_back_used = np.append(times_back_used,time[jj])
    return filename_list, times_back_used

In [7]:
def generate_df(variables, length):
    df = pd.DataFrame()
    for vv in variables:
        df[vv] = np.zeros(length)
    return df

In [8]:
def segments(curve):
    return list(map(LineString, zip(curve.coords[:-1], curve.coords[1:])))

In [9]:
#nearest neighbor interpolation, with a time component to the xarray
#gives x and y values to select
#hrrr needs to be for a given timestep and subsetted to polygon shape
def interpnn_3d(hrrr, perps):
    #code up nearest neighbor
    hrrr_lat_values = hrrr['grid_latt'].values[0,:,:]
    hrrr_lat_shape = hrrr_lat_values.shape
    hrrr_lat = np.tile(hrrr_lat_values,(len(perps),1,1))
    midpoint_lat = np.tile(perps['y_mid'].values,hrrr_lat_shape[0]*hrrr_lat_shape[1]).reshape(hrrr_lat.shape,order='F')
    
    hrrr_lon_values = hrrr['grid_lont'].values[0,:,:]
    hrrr_lon_shape = hrrr_lon_values.shape
    hrrr_lon = np.tile(hrrr_lon_values,(len(perps),1,1))
    midpoint_lon = np.tile(perps['x_mid'].values,hrrr_lon_shape[0]*hrrr_lon_shape[1]).reshape(hrrr_lon.shape,order='F')
        
    distances = np.sqrt(((hrrr_lon-midpoint_lon)**2)+((hrrr_lat-midpoint_lat)**2))
    indices = np.array([np.where(distances==np.nanmin(distances,axis=(1,2))[kk])for kk in range(len(perps))])
    x_nearest_sel = np.squeeze(indices[:,1,:])
    y_nearest_sel = np.squeeze(indices[:,2,:])
    return x_nearest_sel, y_nearest_sel

# Plot Directional HDW (for jan 30th update)

In [ ]:
# process features using the Chen polygons
#2020 fires
#fire_incidents = ['AUGUST COMPLEX','BOBCAT', 'CREEK', 'DOLAN','LAKE']
fire_incidents = ['AUGUST COMPLEX']

start_time=12
path_poly = '/data2/lthapa/ML_daily/fire_polygons/'
suffix_poly = '_Chen_Poly_Perim_Perp.csv'

for jj in range(len(fire_incidents)):
    fire_name = fire_incidents[jj].lower().replace(' ','_')
    #print(path_poly+fire_name+suffix_poly)
    fire = pd.read_csv(path_poly+fire_name+suffix_poly)
    fire=fire.dropna()
    
    for c in ['perimeter','polygon','fireline_split']:
        fire[c] = fire[c].apply(shapely.wkt.loads) #read in the polygons as shapes
    fire['time'] = fire['time'].astype('datetime64')
    
    #get the polygon dataframe set up
    fire_poly = gpd.GeoDataFrame(fire[['time','polygon']],geometry='polygon')
    fire_poly[str(start_time)+ 'Z Start Day'] = (pd.to_datetime(fire_poly['time'].values)-datetime.timedelta(hours=12)).strftime('%Y-%m-%d')

    fire_poly['polygon'] = fire_poly['polygon'].buffer(0.001763092448) #need to buffer because we saved the points, which have no area
    fire_poly = fire_poly.set_crs('EPSG:4326') #set the CRS to lat/lon
    fire_poly = fire_poly.to_crs('EPSG:3347') #transform to LCC

    #dissolve to check daily
    fire_poly['UTC Day'] = (pd.to_datetime(fire_poly['time'].values)).strftime('%Y-%m-%d')
    fire_poly['fire area (ha)'] = fire_poly['polygon'].area/10000 #hectares. from m2
    
    #get the perpindicular lines set up
    fire_split = gpd.GeoDataFrame(fire[['time','fireline_split']],geometry='fireline_split')
    fire_split[str(start_time)+ 'Z Start Day'] = (pd.to_datetime(fire_split['time'].values)-datetime.timedelta(hours=12)).strftime('%Y-%m-%d')
    fire_split = fire_split.iloc[np.array(fire_poly['UTC Day'].values,dtype='datetime64')<=np.datetime64('2020-10-31'),:]
 
    fire_perim  = gpd.GeoDataFrame(fire[['time','perimeter']],geometry='perimeter')
    fire_perim[str(start_time)+ 'Z Start Day'] = (pd.to_datetime(fire_perim['time'].values)-datetime.timedelta(hours=12)).strftime('%Y-%m-%d')
    fire_perim = fire_perim.iloc[np.array(fire_poly['UTC Day'].values,dtype='datetime64')<=np.datetime64('2020-10-31'),:]
    
    fire_poly = fire_poly.iloc[np.array(fire_poly['UTC Day'].values,dtype='datetime64')<=np.datetime64('2020-10-31'),:]
    
    #print(fire_poly)
    #print(fire_split)
    fire_poly.plot(column='12Z Start Day')
    plt.show()
    fire_split.plot(column='12Z Start Day')
    plt.show()
    fire_perim.plot(column='12Z Start Day', alpha=0.1)
    plt.show()
    
    print(fire_poly)
    print(fire_split)
    print(fire_perim)

In [ ]:
time_ave_interval=12
for ii in [4,44]:#len(fire_perim)):
    time = fire_poly['time'].values[ii]
    print(time)
    perim_today = fire_perim[fire_poly['time'].values==time].reset_index()
    split_today = fire_split[fire_poly['time'].values==time].reset_index()

    
    perimeter_bounds = perim_today.buffer(0.05).bounds
    
    #get the HRRR averaged
    times_back = pd.date_range(start=np.datetime64(time), end=np.datetime64(time)+
                                np.timedelta64(11,'h'),freq='H') #do now out to now+11hr
    files_back,times_back_used = make_file_namelist(times_back,'/data2/lthapa/ML_daily/pygraf/Processed_HRRR_YYYYMMDDHH.nc')

    dat_hrrr = xr.open_mfdataset(files_back,concat_dim='time',combine='nested',compat='override', coords='all')
    dat_hrrr = dat_hrrr.assign_coords({'time': times_back_used}) 

    hrrr_subdaily_mean = dat_hrrr.resample(time=str(time_ave_interval)+'H', label='left').mean(dim='time') #take the daily mean         

    time_sel,grid_yt_sel, grid_xt_sel = np.where((hrrr_subdaily_mean['grid_lont']<perimeter_bounds['maxx'].values)&
                                    (hrrr_subdaily_mean['grid_lont']>perimeter_bounds['minx'].values)&
                                    (hrrr_subdaily_mean['grid_latt']<perimeter_bounds['maxy'].values)&
                                    (hrrr_subdaily_mean['grid_latt']>perimeter_bounds['miny'].values))
    hrrr_subdaily_mean_region = hrrr_subdaily_mean.sel(time=time,
                                                       grid_xt=np.unique(grid_xt_sel),
                                                       grid_yt=np.unique(grid_yt_sel))
    #get perpindicular segments
    segs_list = list(split_today.loc[0,'fireline_split'].geoms)
    segs = gpd.GeoDataFrame({'ind':np.arange(0,len(segs_list)),
                        'time': np.repeat(time,len(segs_list)),
                        'geometry':segs_list})
    segs = segs[segs.length>0.0005]
    perp_segs = find_perp_outside_segments(segs,0.005, perim_today.loc[0,'perimeter']) #get the perpindicular segments
    #print(perp_segs)
    #do the wind interpolation
    x_interp, y_interp = interpnn(hrrr_subdaily_mean_region, perp_segs)
    uwind_nearest = hrrr_subdaily_mean_region['uwind_10m'].values[x_interp,y_interp]
    vwind_nearest = hrrr_subdaily_mean_region['vwind_10m'].values[x_interp,y_interp]
    
    #get the length weights
    fireline_length = segs['geometry'].length
    fireline_length_fraction = fireline_length/(np.nansum(fireline_length))
    #print(fireline_length_fraction)
    #get the perpindicular fraction
    #unit_vectors
    uwind_unit = uwind_nearest/np.sqrt((uwind_nearest**2)+(vwind_nearest**2))
    vwind_unit = vwind_nearest/np.sqrt((uwind_nearest**2)+(vwind_nearest**2))
    speed_nearest = np.sqrt((uwind_nearest**2)+(vwind_nearest**2))

    #what portion of the unit vector is along the perpindicular? this uses cos(x) = u.v/||u||*||v|| vector law
    scale_factor = ((perp_segs['x']*uwind_unit)+(perp_segs['y']*vwind_unit))/(np.sqrt((perp_segs['x']**2)+(perp_segs['y']**2)))
    scale_factor_zeroed = ((perp_segs['x']*uwind_unit)+(perp_segs['y']*vwind_unit))/(np.sqrt((perp_segs['x']**2)+(perp_segs['y']**2)))
    scale_factor_zeroed[scale_factor_zeroed<0] =0 
    #print(scale_factor_zeroed)
    
    speed_perp = speed_nearest*scale_factor_zeroed
    print('perp speeds '+str(np.nanmean(speed_perp.values)))
    print('perp speeds, length weighted '+str(np.nansum(speed_perp.values*fireline_length_fraction)))
    print('nearest speeds '+str(np.nanmean(speed_nearest)))
    print('average direction: ' + str(np.nanmean(scale_factor)))
    print('average direction (abs val): ' + str(np.nanmean(scale_factor_zeroed)))

    """the above is based on 
    hrrr_derived.loc[count, 'hd0w0_perp_out'] = hd00*(np.nanmean(speed_nearest*scale_factor_zeroed))
    df_hrrr_derived.loc[count, 'hd0w0_perp_out_lw'] = hd00*(np.nansum(speed_nearest*scale_factor_zeroed*fireline_length_fraction))
    """
    uwind_perp = (perp_segs['x']/(np.sqrt((perp_segs['x']**2)+(perp_segs['y']**2))))*speed_perp
    vwind_perp = (perp_segs['y']/(np.sqrt((perp_segs['x']**2)+(perp_segs['y']**2))))*speed_perp
    
    #u_perp = perp_segs['x']
    #v_perp = perp_segs['y']

    #make the plots
    fig,ax = plt.subplots(figsize=(10,10))
    plt.quiver(hrrr_subdaily_mean_region['grid_lont'], hrrr_subdaily_mean_region['grid_latt'], 
               hrrr_subdaily_mean_region['uwind_10m'], hrrr_subdaily_mean_region['vwind_10m'],alpha=0.3)
    perim_today.plot(ax=ax)
    split_today.plot(ax=ax,color='r')
    if ii>0: #if past 1st timestep of fire
        fire_perim[fire_poly['time']==time-np.timedelta64(12,'h')].plot(ax=ax,color='k',alpha=0.5)
        fire_perim[fire_poly['time']==time+np.timedelta64(12,'h')].plot(ax=ax,color='k',alpha=0.2)

    
    plt.quiver(perp_segs['x_mid'], perp_segs['y_mid'], uwind_nearest, vwind_nearest, color='r',alpha=0.5)
    plt.quiver(perp_segs['x_mid'], perp_segs['y_mid'], uwind_perp, vwind_perp, color='g',alpha=0.5)
    #plt.quiver(perp_segs['x_mid'], perp_segs['y_mid'], perp_segs['x'], perp_segs['y'], color='m',alpha = 0.5)

    plt.show()
    


### Feb 6th update, plot winds hourly for each polygon

In [ ]:
time_ave_interval=12
for ii in [4,44]:#len(fire_perim)):
    time = fire_poly['time'].values[ii]
    print(time)
    perim_today = fire_perim[fire_poly['time'].values==time].reset_index()
    split_today = fire_split[fire_poly['time'].values==time].reset_index()
    
    perimeter_bounds = perim_today.buffer(0.05).bounds
    
    #get the HRRR averaged
    times_back = pd.date_range(start=np.datetime64(time), end=np.datetime64(time)+
                                np.timedelta64(11,'h'),freq='H') #do now out to now+11hr
    files_back,times_back_used = make_file_namelist(times_back,'/data2/lthapa/ML_daily/pygraf/Processed_HRRR_YYYYMMDDHH.nc')

    dat_hrrr = xr.open_mfdataset(files_back,concat_dim='time',combine='nested',compat='override', coords='all')
    dat_hrrr = dat_hrrr.assign_coords({'time': times_back_used}) 

    # we want all of the hours, so we only select by region
    time_sel,grid_yt_sel, grid_xt_sel = np.where((dat_hrrr['grid_lont']<perimeter_bounds['maxx'].values)&
                                    (dat_hrrr['grid_lont']>perimeter_bounds['minx'].values)&
                                    (dat_hrrr['grid_latt']<perimeter_bounds['maxy'].values)&
                                    (dat_hrrr['grid_latt']>perimeter_bounds['miny'].values))
    hrrr_region = dat_hrrr.sel(time=times_back_used,
                                grid_xt=np.unique(grid_xt_sel),
                                grid_yt=np.unique(grid_yt_sel))

    print(hrrr_region)
    #get perpindicular segments
    segs_list = list(split_today.loc[0,'fireline_split'].geoms)
    segs = gpd.GeoDataFrame({'ind':np.arange(0,len(segs_list)),
                        'time': np.repeat(time,len(segs_list)),
                        'geometry':segs_list})
    segs = segs[segs.length>0.0005]
    perp_segs = find_perp_outside_segments(segs,0.005, perim_today.loc[0,'perimeter']) #get the perpindicular segments
   
    #get the length weights
    fireline_length = segs['geometry'].length
    fireline_length_fraction = fireline_length/(np.nansum(fireline_length))

    #do the wind interpolation
    x_interp, y_interp = interpnn_3d(hrrr_region, perp_segs)
    
    for hour in times_back_used:
        print(hour)
        uwind_nearest = hrrr_region['uwind_10m'].sel(time=hour).values[x_interp,y_interp] 
        vwind_nearest = hrrr_region['vwind_10m'].sel(time=hour).values[x_interp,y_interp]

        #get the perpindicular fraction
        #unit_vectors
        uwind_unit = uwind_nearest/np.sqrt((uwind_nearest**2)+(vwind_nearest**2))
        vwind_unit = vwind_nearest/np.sqrt((uwind_nearest**2)+(vwind_nearest**2))
        speed_nearest = np.sqrt((uwind_nearest**2)+(vwind_nearest**2))

        #what portion of the unit vector is along the perpindicular? this uses cos(x) = u.v/||u||*||v|| vector law
        scale_factor = ((perp_segs['x']*uwind_unit)+(perp_segs['y']*vwind_unit))/(np.sqrt((perp_segs['x']**2)+(perp_segs['y']**2)))
        scale_factor_zeroed = ((perp_segs['x']*uwind_unit)+(perp_segs['y']*vwind_unit))/(np.sqrt((perp_segs['x']**2)+(perp_segs['y']**2)))
        scale_factor_zeroed[scale_factor_zeroed<0] =0 
        #print(scale_factor_zeroed)
    
        speed_perp = speed_nearest*scale_factor_zeroed
        print('perp speeds '+str(np.nanmean(speed_perp.values)))
        print('perp speeds, length weighted '+str(np.nansum(speed_perp.values*fireline_length_fraction)))
        print('nearest speeds '+str(np.nanmean(speed_nearest)))
        print('average direction: ' + str(np.nanmean(scale_factor)))
        print('average direction (abs val): ' + str(np.nanmean(scale_factor_zeroed)))

        """the above is based on 
        hrrr_derived.loc[count, 'hd0w0_perp_out'] = hd00*(np.nanmean(speed_nearest*scale_factor_zeroed))
        df_hrrr_derived.loc[count, 'hd0w0_perp_out_lw'] = hd00*(np.nansum(speed_nearest*scale_factor_zeroed*fireline_length_fraction))
        """
        uwind_perp = (perp_segs['x']/(np.sqrt((perp_segs['x']**2)+(perp_segs['y']**2))))*speed_perp
        vwind_perp = (perp_segs['y']/(np.sqrt((perp_segs['x']**2)+(perp_segs['y']**2))))*speed_perp

        #u_perp = perp_segs['x']
        #v_perp = perp_segs['y']

        #make the plots
        fig,ax = plt.subplots(figsize=(10,10))
        plt.quiver(hrrr_region['grid_lont'].values[0,:,:], hrrr_region['grid_latt'].values[0,:,:], 
                   hrrr_region['uwind_10m'].values[0,:,:], hrrr_region['vwind_10m'].values[0,:,:],alpha=0.3)
        perim_today.plot(ax=ax)
        split_today.plot(ax=ax,color='r')
        if ii>0: #if past 1st timestep of fire
            fire_perim[fire_poly['time']==time-np.timedelta64(12,'h')].plot(ax=ax,color='k',alpha=0.5)
            fire_perim[fire_poly['time']==time+np.timedelta64(12,'h')].plot(ax=ax,color='k',alpha=0.2)


        #plt.quiver(perp_segs['x_mid'], perp_segs['y_mid'], uwind_nearest, vwind_nearest, color='r',alpha=0.5)
        plt.quiver(perp_segs['x_mid'], perp_segs['y_mid'], uwind_perp, vwind_perp, color='g',alpha=0.5)
        #plt.quiver(perp_segs['x_mid'], perp_segs['y_mid'], perp_segs['x'], perp_segs['y'], color='m',alpha = 0.5)

        plt.show()
    
    

In [ ]:
#test intersection to see if it would be easy to implement
time_ave_interval=12
for ii in [4,44]:#len(fire_perim)):
    time = fire_poly['time'].values[ii]
    print(time)
    perim_today = fire_perim[fire_poly['time'].values==time].reset_index()
    split_today = fire_split[fire_poly['time'].values==time].reset_index()

    
    perimeter_bounds = perim_today.buffer(0.05).bounds
    
    #get the HRRR averaged
    times_back = pd.date_range(start=np.datetime64(time), end=np.datetime64(time)+
                                np.timedelta64(11,'h'),freq='H') #do now out to now+11hr
    files_back,times_back_used = make_file_namelist(times_back,'/data2/lthapa/ML_daily/pygraf/Processed_HRRR_YYYYMMDDHH.nc')

    dat_hrrr = xr.open_mfdataset(files_back,concat_dim='time',combine='nested',compat='override', coords='all')
    dat_hrrr = dat_hrrr.assign_coords({'time': times_back_used}) 

    hrrr_subdaily_mean = dat_hrrr.resample(time=str(time_ave_interval)+'H', label='left').mean(dim='time') #take the daily mean         

    time_sel,grid_yt_sel, grid_xt_sel = np.where((hrrr_subdaily_mean['grid_lont']<perimeter_bounds['maxx'].values)&
                                    (hrrr_subdaily_mean['grid_lont']>perimeter_bounds['minx'].values)&
                                    (hrrr_subdaily_mean['grid_latt']<perimeter_bounds['maxy'].values)&
                                    (hrrr_subdaily_mean['grid_latt']>perimeter_bounds['miny'].values))
    hrrr_subdaily_mean_region = hrrr_subdaily_mean.sel(time=time,
                                                       grid_xt=np.unique(grid_xt_sel),
                                                       grid_yt=np.unique(grid_yt_sel))
    #get perpindicular segments
    segs_list = list(split_today.loc[0,'fireline_split'].geoms)
    segs = gpd.GeoDataFrame({'ind':np.arange(0,len(segs_list)),
                        'time': np.repeat(time,len(segs_list)),
                        'geometry':segs_list})
    segs = segs[segs.length>0.0005]
    perp_segs = find_perp_outside_segments(segs,0.005, perim_today.loc[0,'perimeter']) #get the perpindicular segments
    #print(perp_segs)
    
    if ii>0: #if past 1st timestep of fire
        perim_yesterday = fire_perim[fire_poly['time']==time-np.timedelta64(12,'h')].reset_index()
        for jj in range(len(segs)):
            seg = segs.loc[ii,'geometry']
            print(seg)
            print(perim_yesterday.loc[0,'perimeter'].buffer(0.001).covered_by(seg))

## Check that the buffers work

In [ ]:
#all fires
#fire_incidents = ['BOBCAT', 'DOLAN', 'HOLIDAY FARM','CREEK', 'LAKE', 'CAMERON PEAK', 'PINE GULCH', 'WILLIAMS FLATS', 'SHADY','PEDRO MOUNTAIN', 'WALKER', '204 COW']

#2020 fires
#fire_incidents = ['AUGUST COMPLEX','BOBCAT', 'DOLAN', 'HOLIDAY FARM','CREEK', 'LAKE', 'CAMERON PEAK', 'PINE GULCH']
#fire_incidents=['BOBCAT', 'DOLAN', 'HOLIDAY FARM','CREEK', 'LAKE', 'CAMERON PEAK', 'PINE GULCH']
fire_incidents = ['LAKE']

path_poly = '/data2/lthapa/ML_daily/fire_polygons/'
suffix_poly = 'Z_day_start.geojson'
start_time=12
for jj in range(len(fire_incidents)):
    fire_name = fire_incidents[jj].lower().replace(' ','_')
    print(path_poly+fire_name+'_VIIRS_daily_'+str(start_time)+suffix_poly)
    fire_daily = gpd.read_file(path_poly+fire_name+'_VIIRS_daily_'+str(start_time)+suffix_poly)
    fire_daily=fire_daily.drop(columns=['Current Overpass'])
    fire_daily = fire_daily.drop(np.where(fire_daily['geometry']==None)[0])
    fire_daily['fire area (ha)'] = fire_daily['geometry'].area/10000 #hectares
    fire_daily.set_geometry(col='geometry', inplace=True) #designate the geometry column
    fire_daily = fire_daily.rename(columns={'Current Day':'UTC Day', 'Local Day': str(start_time)+ 'Z Start Day'})
    
    fire_daily = fire_daily.iloc[np.array(fire_daily['UTC Day'].values,dtype='datetime64')<=np.datetime64('2020-10-31'),:]

    #print(fire_daily)
    
    #merra
    tic = time.time()
    merra_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (fire_daily.iloc[ii:ii+1],'ESI_GRID',0.05) 
                                 for ii in range(len(fire_daily)))
    toc = time.time()
    print(toc-tic)
    fire_merra_intersection=gpd.GeoDataFrame(pd.concat(merra_intersections, ignore_index=True))
    fire_merra_intersection.set_geometry(col='geometry')
    fire_merra_intersection = fire_merra_intersection.set_index([str(start_time)+ 'Z Start Day', 'lat', 'lon'])
    print(fire_merra_intersection)
    fire_merra_intersection_xr = fire_merra_intersection.to_xarray()
    print(fire_merra_intersection_xr['weights'].sum(dim=['lat','lon']).values)
    

    esi_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (fire_daily.iloc[ii:ii+1],'ESI_GRID',0.05) 
                                 for ii in range(len(fire_daily)))
    toc = time.time()
    fire_esi_intersection=gpd.GeoDataFrame(pd.concat(esi_intersections, ignore_index=True))
    fire_esi_intersection.set_geometry(col='geometry')
    fire_esi_intersection = fire_esi_intersection.set_index([str(start_time)+ 'Z Start Day', 'lat', 'lon'])
    fire_esi_intersection_xr = fire_esi_intersection.to_xarray()
    print(fire_esi_intersection_xr['weights'].sum(dim=['lat','lon']))

# Testing

In [ ]:
start_time=12
fire_daily = gpd.read_file('./fire_polygons/cameron_peak_VIIRS_daily_12Z_day_start.geojson')
#fire_daily = gpd.read_file('./fire_polygons/august_complex_VIIRS_daily_12Z_day_start.geojson')
#fire_daily = gpd.read_file('./fire_polygons/williams_flats_VIIRS_daily_12Z_day_start.geojson')
fire_daily=fire_daily.drop(columns=['Current Overpass'])
fire_daily = fire_daily.drop(np.where(fire_daily['geometry']==None)[0])
fire_daily['fire area (ha)'] = fire_daily['geometry'].area/10000 #hectares
fire_daily.set_geometry(col='geometry', inplace=True) #designate the geometry column
fire_daily = fire_daily.rename(columns={'Current Day':'UTC Day', 'Local Day': str(start_time)+ 'Z Start Day'})
fire_daily = fire_daily.iloc[np.array(fire_daily['UTC Day'].values,dtype='datetime64')<=np.datetime64('2020-10-31'),:]
print(fire_daily)

In [ ]:
#hrrr_timeseries(fire_daily.iloc[15:25],12)

times_intersect = np.unique(fire_daily[str(12)+ 'Z Start Day'].iloc[39:40].values)
for today in times_intersect:
    print(today)
    times_back = pd.date_range(start=np.datetime64(today)-np.timedelta64(5,'D'), end=np.datetime64(today)+
                                   np.timedelta64(1,'D'),freq='H')
    files_back,times_back_used = make_hrrr_file_namelist(times_back)
    
    #print(files_back)
    dat_hrrr = xr.open_mfdataset(files_back,concat_dim='time',combine='nested',compat='override', coords='all')
    dat_hrrr = dat_hrrr.assign_coords({'time': times_back_used})
    hrrr_daily_mean = dat_hrrr.resample(time='24H',base=12, label='left').mean(dim='time') #take the daily mean        

    print(hrrr_daily_mean.time)
    dat_hrrr.close()

In [ ]:
ints = calculate_intersection(fire_daily.iloc[11:12],'ESI_GRID',0.05)
print(ints.sum())
#tic = time.time()
#hrrr_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
#                                 (fire_daily.iloc[ii:ii+1],'HRRR_GRID',0.01) 
#                                 for ii in range(len(fire_daily)))
#toc=time.time()
#print(toc-tic)

In [ ]:
grid = xr.open_dataset('PWS_GRID.nc')
print(grid.LAT_CTR[0:5,0:5])

In [ ]:
grid = xr.open_dataset('HRRR_GRID.nc')
bf = 0.05

for ii in range(len(fire_daily)):
    poly=fire_daily[ii:ii+1]
    #get the bounds of the buffered polygons
    poly_latlon =poly.to_crs(epsg=4326)
    bounds = poly_latlon.buffer(bf).bounds
    
    #first check for rows and cols
    [rows,cols] = np.where((grid.LAT_CTR.values>bounds['miny'].values)&
                    (grid.LAT_CTR.values<bounds['maxy'].values)&
                    (grid.LON_CTR.values>bounds['minx'].values)&
                    (grid.LON_CTR.values<bounds['maxx'].values))
    print(rows,cols)